# Система прогнозирования продаж и планирования отгрузок
## Wildberries & Ozon

**Цель:** Построить систему прогнозирования продаж и планирования отгрузок с rolling-прогнозом на 18 месяцев вперёд по каждому продукту (solo-code).

**Период:** 26.03.2024 – 20.12.2025  
**Частота:** Дневная


In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Прогнозирование
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Библиотеки загружены")


## 1. Загрузка данных

Поддерживаются форматы CSV и Excel. Укажите пути к файлам ниже.


In [ ]:
# ============================================
# НАСТРОЙКА ПУТЕЙ К ФАЙЛАМ
# ============================================
# Укажите пути к вашим файлам или оставьте примеры для тестирования

FILE_PATHS = {
    'wb_sales': 'wb_sales.csv',  # Дата, solo-code, SKU, Количество упак.
    'ozon_sales': 'ozon_sales.csv',  # Дата, solo-code, SKU, Количество упак.
    'historical_shipment': 'historical_shipment.csv',  # solo-code, Дата, Количества
    'wb_stocks': 'wb_stocks.csv',  # Дата, Склад, solo-code, SKU, Остаток
    'ozon_stocks': 'ozon_stocks.csv',  # Дата, Склад, solo-code, SKU, Остаток
    'our_stocks': 'our_stocks.csv',  # Дата, solo-code, SKU, Остаток
    'withdraw': 'withdraw.csv',  # solo-code, SKU
    'defecture': 'defecture.csv',  # solo-code, SKU, Дата окончания дефектуры
    'count_box': 'count_box.csv'  # solo-code, Кол-во
}

def load_file(file_path, file_type='csv'):
    """Загрузка файла CSV или Excel"""
    try:
        if file_type == 'csv':
            df = pd.read_csv(file_path, encoding='utf-8')
        else:
            df = pd.read_excel(file_path)
        return df
    except FileNotFoundError:
        print(f"⚠️ Файл {file_path} не найден. Будет создан тестовый датасет.")
        return None
    except Exception as e:
        print(f"❌ Ошибка при загрузке {file_path}: {e}")
        return None

# Загрузка данных
print("📂 Загрузка данных...")
data = {}

for key, path in FILE_PATHS.items():
    # Пробуем CSV
    df = load_file(path, 'csv')
    if df is None:
        # Пробуем Excel
        excel_path = path.replace('.csv', '.xlsx')
        df = load_file(excel_path, 'excel')
    
    if df is not None:
        data[key] = df
        print(f"✅ {key}: {len(df)} строк, колонки: {list(df.columns)}")
    else:
        print(f"⚠️ {key}: файл не найден, будет создан тестовый датасет")

print(f"\n📊 Загружено файлов: {len([k for k, v in data.items() if v is not None])}")


In [ ]:
# Создание тестовых данных, если файлы не загружены
def create_sample_data():
    """Создание тестовых данных для демонстрации"""
    np.random.seed(42)
    dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
    solo_codes = ['SC001', 'SC002', 'SC003']
    skus = {sc: [f'{sc}_SKU{i}' for i in range(1, 4)] for sc in solo_codes}
    
    sample_data = {}
    
    # Продажи WB
    wb_sales = []
    for date in dates:
        for sc in solo_codes:
            for sku in skus[sc]:
                qty = max(0, int(np.random.poisson(10) * (1 + 0.3 * np.sin(date.dayofyear / 365 * 2 * np.pi))))
                wb_sales.append({'Дата': date, 'solo-code': sc, 'SKU': sku, 'Количество упак.': qty})
    sample_data['wb_sales'] = pd.DataFrame(wb_sales)
    
    # Продажи Ozon
    ozon_sales = []
    for date in dates:
        for sc in solo_codes:
            for sku in skus[sc]:
                qty = max(0, int(np.random.poisson(8) * (1 + 0.2 * np.sin(date.dayofyear / 365 * 2 * np.pi))))
                ozon_sales.append({'Дата': date, 'solo-code': sc, 'SKU': sku, 'Количество упак.': qty})
    sample_data['ozon_sales'] = pd.DataFrame(ozon_sales)
    
    # Исторические отгрузки
    shipments = []
    for sc in solo_codes:
        shipment_dates = dates[::30]  # Примерно раз в месяц
        for date in shipment_dates:
            shipments.append({
                'solo-code': sc,
                'Дата': date,
                'Количества': int(np.random.uniform(100, 500))
            })
    sample_data['historical_shipment'] = pd.DataFrame(shipments)
    
    # Остатки
    for key in ['wb_stocks', 'ozon_stocks', 'our_stocks']:
        stocks = []
        for date in dates[::3]:  # Каждые 3 дня
            for sc in solo_codes:
                for sku in skus[sc]:
                    stock = max(0, int(np.random.uniform(50, 300)))
                    row = {'Дата': date, 'solo-code': sc, 'SKU': sku, 'Остаток': stock}
                    if key in ['wb_stocks', 'ozon_stocks']:
                        row['Склад'] = 'WH1'
                    stocks.append(row)
        sample_data[key] = pd.DataFrame(stocks)
    
    # Справочники
    sample_data['withdraw'] = pd.DataFrame({
        'solo-code': ['SC003'],
        'SKU': ['SC003_SKU1']
    })
    
    sample_data['defecture'] = pd.DataFrame({
        'solo-code': ['SC002'],
        'SKU': ['SC002_SKU2'],
        'Дата окончания дефектуры': ['2024-06-15']
    })
    
    sample_data['count_box'] = pd.DataFrame({
        'solo-code': solo_codes,
        'Кол-во': [12, 24, 18]
    })
    
    return sample_data

# Если данные не загружены, создаем тестовые
if len(data) == 0 or all(v is None for v in data.values()):
    print("📝 Создание тестовых данных...")
    data = create_sample_data()
    print("✅ Тестовые данные созданы")

# Обновляем data, заполняя None значения
for key in FILE_PATHS.keys():
    if key not in data or data.get(key) is None:
        if 'sample_data' not in locals():
            sample_data = create_sample_data()
        data[key] = sample_data.get(key, pd.DataFrame())

print("\n📋 Структура данных:")
for key, df in data.items():
    if df is not None and len(df) > 0:
        print(f"{key}: {len(df)} строк")


## 2. Подготовка данных

### 2.1. Создание календаря


In [ ]:
# Создание полного дневного календаря
START_DATE = pd.to_datetime('2024-03-26')
END_DATE = pd.to_datetime('2025-12-20')
FULL_CALENDAR = pd.date_range(START_DATE, END_DATE, freq='D')

print(f"📅 Календарь создан: {len(FULL_CALENDAR)} дней")
print(f"   Период: {START_DATE.date()} - {END_DATE.date()}")

# Получаем все solo-code из данных
all_solo_codes = set()
for key, df in data.items():
    if df is not None and 'solo-code' in df.columns:
        all_solo_codes.update(df['solo-code'].unique())

all_solo_codes = sorted(list(all_solo_codes))
print(f"\n📦 Найдено solo-code: {len(all_solo_codes)}")
print(f"   Примеры: {all_solo_codes[:5]}")


### 2.2. Обработка продаж


In [ ]:
def prepare_sales_data(sales_df, marketplace_name, date_col='Дата', solo_code_col='solo-code', 
                       sku_col='SKU', qty_col='Количество упак.'):
    """Подготовка данных о продажах: приведение к дневной частоте, заполнение пропусков"""
    if sales_df is None or len(sales_df) == 0:
        return None
    
    # Нормализация названий колонок
    sales_df = sales_df.copy()
    sales_df[date_col] = pd.to_datetime(sales_df[date_col])
    
    # Агрегация по дате и solo-code (суммируем по SKU)
    sales_agg = sales_df.groupby([date_col, solo_code_col])[qty_col].sum().reset_index()
    sales_agg.columns = ['date', 'solo_code', 'sales']
    
    # Создание полного календаря для каждого solo-code
    calendar_df = pd.DataFrame({
        'date': FULL_CALENDAR
    })
    
    result_list = []
    for solo_code in all_solo_codes:
        # Фильтруем данные по solo-code
        sc_data = sales_agg[sales_agg['solo_code'] == solo_code].copy()
        
        # Объединяем с календарем
        sc_calendar = calendar_df.copy()
        sc_calendar['solo_code'] = solo_code
        sc_calendar = sc_calendar.merge(
            sc_data[['date', 'sales']], 
            on='date', 
            how='left'
        )
        sc_calendar['sales'] = sc_calendar['sales'].fillna(0)
        
        result_list.append(sc_calendar)
    
    result = pd.concat(result_list, ignore_index=True)
    result['marketplace'] = marketplace_name
    
    return result

# Обработка продаж WB и Ozon
print("📊 Обработка продаж...")
wb_sales_clean = prepare_sales_data(data.get('wb_sales'), 'WB')
ozon_sales_clean = prepare_sales_data(data.get('ozon_sales'), 'Ozon')

# Объединение продаж
all_sales = pd.concat([wb_sales_clean, ozon_sales_clean], ignore_index=True)

# Total sales (сумма по маркетплейсам)
total_sales = all_sales.groupby(['date', 'solo_code'])['sales'].sum().reset_index()
total_sales['marketplace'] = 'Total'

# Финальный датасет продаж
sales_final = pd.concat([all_sales, total_sales], ignore_index=True)

print(f"✅ Продажи обработаны:")
print(f"   WB: {len(wb_sales_clean)} записей")
print(f"   Ozon: {len(ozon_sales_clean)} записей")
print(f"   Всего: {len(sales_final)} записей")

# Пример восстановленных рядов
print("\n📈 Пример восстановленных рядов (первые 10 дней для первого solo-code):")
example_sc = all_solo_codes[0]
example_data = sales_final[
    (sales_final['solo_code'] == example_sc) & 
    (sales_final['marketplace'] == 'Total')
].head(10)
print(example_data[['date', 'sales']].to_string(index=False))


### 2.3. Обработка остатков


In [ ]:
def prepare_stocks_data(stocks_df, stock_type, date_col='Дата', solo_code_col='solo-code', 
                        sku_col='SKU', stock_col='Остаток'):
    """Подготовка данных об остатках: forward fill, backward fill, проверка аномалий"""
    if stocks_df is None or len(stocks_df) == 0:
        return None
    
    stocks_df = stocks_df.copy()
    stocks_df[date_col] = pd.to_datetime(stocks_df[date_col])
    
    # Агрегация по дате и solo-code (суммируем по SKU и складам)
    stocks_agg = stocks_df.groupby([date_col, solo_code_col])[stock_col].sum().reset_index()
    stocks_agg.columns = ['date', 'solo_code', 'stock']
    
    # Создание полного календаря для каждого solo-code
    result_list = []
    for solo_code in all_solo_codes:
        sc_data = stocks_agg[stocks_agg['solo_code'] == solo_code].copy()
        
        if len(sc_data) == 0:
            continue
        
        # Объединяем с календарем
        sc_calendar = pd.DataFrame({'date': FULL_CALENDAR})
        sc_calendar = sc_calendar.merge(sc_data, on='date', how='left')
        
        # Forward fill
        sc_calendar['stock'] = sc_calendar['stock'].ffill()
        
        # Backward fill для начала ряда
        sc_calendar['stock'] = sc_calendar['stock'].bfill()
        
        # Если все еще есть пропуски, заполняем нулями
        sc_calendar['stock'] = sc_calendar['stock'].fillna(0)
        
        # Проверка аномалий (отрицательные значения)
        sc_calendar['stock'] = sc_calendar['stock'].clip(lower=0)
        
        sc_calendar['solo_code'] = solo_code
        result_list.append(sc_calendar)
    
    if not result_list:
        return None
    
    result = pd.concat(result_list, ignore_index=True)
    result['stock_type'] = stock_type
    
    return result

# Обработка остатков
print("📦 Обработка остатков...")
wb_stocks_clean = prepare_stocks_data(data.get('wb_stocks'), 'WB')
ozon_stocks_clean = prepare_stocks_data(data.get('ozon_stocks'), 'Ozon')
our_stocks_clean = prepare_stocks_data(data.get('our_stocks'), 'Our')

# Объединение
all_stocks = pd.concat([df for df in [wb_stocks_clean, ozon_stocks_clean, our_stocks_clean] 
                       if df is not None], ignore_index=True)

print(f"✅ Остатки обработаны:")
if wb_stocks_clean is not None:
    print(f"   WB: {len(wb_stocks_clean)} записей")
if ozon_stocks_clean is not None:
    print(f"   Ozon: {len(ozon_stocks_clean)} записей")
if our_stocks_clean is not None:
    print(f"   Our: {len(our_stocks_clean)} записей")

# Статистика пропусков
if wb_stocks_clean is not None:
    print("\n📊 Статистика пропусков (до обработки):")
    for sc in all_solo_codes[:3]:
        sc_data = wb_stocks_clean[wb_stocks_clean['solo_code'] == sc]
        if len(sc_data) > 0:
            original_dates = set(data.get('wb_stocks', pd.DataFrame()).get('Дата', []))
            if len(original_dates) > 0:
                original_dates = set(pd.to_datetime(list(original_dates)))
                total_days = len(FULL_CALENDAR)
                available_days = len([d for d in FULL_CALENDAR if d in original_dates])
                print(f"   {sc}: {available_days}/{total_days} дней ({100*available_days/total_days:.1f}%)")


### 2.4. Обработка отгрузок


In [ ]:
def prepare_shipments_data(shipments_df, date_col='Дата', solo_code_col='solo-code', qty_col='Количества'):
    """Подготовка данных об отгрузках: приведение к дневной частоте"""
    if shipments_df is None or len(shipments_df) == 0:
        return None
    
    shipments_df = shipments_df.copy()
    shipments_df[date_col] = pd.to_datetime(shipments_df[date_col])
    
    # Агрегация по дате и solo-code
    shipments_agg = shipments_df.groupby([date_col, solo_code_col])[qty_col].sum().reset_index()
    shipments_agg.columns = ['date', 'solo_code', 'shipment']
    
    # Создание полного календаря для каждого solo-code
    result_list = []
    for solo_code in all_solo_codes:
        sc_data = shipments_agg[shipments_agg['solo_code'] == solo_code].copy()
        
        sc_calendar = pd.DataFrame({'date': FULL_CALENDAR})
        sc_calendar = sc_calendar.merge(sc_data, on='date', how='left')
        sc_calendar['shipment'] = sc_calendar['shipment'].fillna(0)
        sc_calendar['solo_code'] = solo_code
        
        result_list.append(sc_calendar)
    
    result = pd.concat(result_list, ignore_index=True)
    return result

# Обработка отгрузок
print("🚚 Обработка отгрузок...")
shipments_clean = prepare_shipments_data(data.get('historical_shipment'))

if shipments_clean is not None:
    print(f"✅ Отгрузки обработаны: {len(shipments_clean)} записей")
    
    # Визуализация: отгрузки vs продажи
    if len(all_solo_codes) > 0:
        example_sc = all_solo_codes[0]
        sc_shipments = shipments_clean[shipments_clean['solo_code'] == example_sc]
        sc_sales = total_sales[total_sales['solo_code'] == example_sc]
        
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # Отгрузки vs продажи
        ax1 = axes[0]
        ax1.plot(sc_shipments['date'], sc_shipments['shipment'], 
                label='Отгрузки', marker='o', markersize=4, alpha=0.7)
        ax1.plot(sc_sales['date'], sc_sales['sales'], 
                label='Продажи (Total)', alpha=0.7)
        ax1.set_title(f'Отгрузки vs Продажи: {example_sc}')
        ax1.set_xlabel('Дата')
        ax1.set_ylabel('Количество')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Интервалы между отгрузками
        ax2 = axes[1]
        shipment_dates = sc_shipments[sc_shipments['shipment'] > 0]['date'].values
        if len(shipment_dates) > 1:
            intervals = np.diff(shipment_dates).astype('timedelta64[D]').astype(int)
            ax2.bar(range(len(intervals)), intervals, alpha=0.7)
            ax2.set_title(f'Интервалы между отгрузками: {example_sc}')
            ax2.set_xlabel('Номер интервала')
            ax2.set_ylabel('Дней')
            ax2.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 Статистика интервалов между отгрузками ({example_sc}):")
        if len(shipment_dates) > 1:
            print(f"   Средний интервал: {np.mean(intervals):.1f} дней")
            print(f"   Медианный интервал: {np.median(intervals):.1f} дней")
            print(f"   Минимальный: {np.min(intervals)} дней")
            print(f"   Максимальный: {np.max(intervals)} дней")
else:
    print("⚠️ Данные об отгрузках отсутствуют")


## 3. EDA и анализ покрытия


In [ ]:
# Расчет дней покрытия (Days Cover)
def calculate_days_cover(sales_df, stocks_wb, stocks_ozon):
    """Расчет дней покрытия запасами"""
    result_list = []
    
    for solo_code in all_solo_codes:
        # Продажи
        sc_sales = sales_df[
            (sales_df['solo_code'] == solo_code) & 
            (sales_df['marketplace'] == 'Total')
        ].copy()
        
        if len(sc_sales) == 0:
            continue
        
        # Средний дневной спрос
        avg_daily_sales = sc_sales['sales'].mean()
        if avg_daily_sales == 0:
            continue
        
        # Остатки WB
        sc_stocks_wb = None
        if stocks_wb is not None:
            sc_stocks_wb = stocks_wb[stocks_wb['solo_code'] == solo_code].copy()
        
        # Остатки Ozon
        sc_stocks_ozon = None
        if stocks_ozon is not None:
            sc_stocks_ozon = stocks_ozon[stocks_ozon['solo_code'] == solo_code].copy()
        
        # Объединение по датам
        cover_df = sc_sales[['date', 'sales']].copy()
        
        if sc_stocks_wb is not None:
            cover_df = cover_df.merge(
                sc_stocks_wb[['date', 'stock']], 
                on='date', 
                how='left', 
                suffixes=('', '_wb')
            )
            cover_df['stock_wb'] = cover_df['stock'].fillna(0)
            cover_df = cover_df.drop('stock', axis=1)
        else:
            cover_df['stock_wb'] = 0
        
        if sc_stocks_ozon is not None:
            cover_df = cover_df.merge(
                sc_stocks_ozon[['date', 'stock']], 
                on='date', 
                how='left', 
                suffixes=('', '_ozon')
            )
            cover_df['stock_ozon'] = cover_df['stock'].fillna(0)
            cover_df = cover_df.drop('stock', axis=1)
        else:
            cover_df['stock_ozon'] = 0
        
        # Days Cover
        cover_df['total_stock'] = cover_df['stock_wb'] + cover_df['stock_ozon']
        cover_df['days_cover'] = cover_df['total_stock'] / avg_daily_sales
        cover_df['days_cover'] = cover_df['days_cover'].replace([np.inf, -np.inf], 0)
        cover_df['solo_code'] = solo_code
        
        # OOS периоды (out of stock)
        cover_df['is_oos'] = (cover_df['total_stock'] == 0) & (cover_df['sales'] > 0)
        
        result_list.append(cover_df)
    
    if not result_list:
        return None
    
    return pd.concat(result_list, ignore_index=True)

# Расчет покрытия
print("📊 Расчет дней покрытия...")
days_cover_df = calculate_days_cover(sales_final, wb_stocks_clean, ozon_stocks_clean)

if days_cover_df is not None:
    print(f"✅ Расчет завершен: {len(days_cover_df)} записей")
    
    # Визуализация для примера
    if len(all_solo_codes) > 0:
        example_sc = all_solo_codes[0]
        sc_cover = days_cover_df[days_cover_df['solo_code'] == example_sc]
        
        fig, axes = plt.subplots(3, 1, figsize=(14, 12))
        
        # Продажи vs отгрузки
        ax1 = axes[0]
        ax1.plot(sc_cover['date'], sc_cover['sales'], label='Продажи', alpha=0.7)
        if shipments_clean is not None:
            sc_ship = shipments_clean[shipments_clean['solo_code'] == example_sc]
            ax1.bar(sc_ship['date'], sc_ship['shipment'], 
                   label='Отгрузки', alpha=0.5, width=1)
        ax1.set_title(f'Продажи vs Отгрузки: {example_sc}')
        ax1.set_ylabel('Количество')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Динамика покрытия
        ax2 = axes[1]
        ax2.plot(sc_cover['date'], sc_cover['days_cover'], 
                label='Days Cover', color='green', alpha=0.7)
        ax2.axhline(y=30, color='r', linestyle='--', label='Минимум 30 дней', alpha=0.5)
        ax2.set_title(f'Динамика покрытия: {example_sc}')
        ax2.set_ylabel('Дней покрытия')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # OOS периоды
        ax3 = axes[2]
        oos_periods = sc_cover[sc_cover['is_oos']]
        if len(oos_periods) > 0:
            ax3.scatter(oos_periods['date'], oos_periods['sales'], 
                       color='red', marker='x', s=100, label='OOS')
        ax3.plot(sc_cover['date'], sc_cover['sales'], 
                label='Продажи', alpha=0.5)
        ax3.set_title(f'OOS периоды: {example_sc}')
        ax3.set_xlabel('Дата')
        ax3.set_ylabel('Продажи')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Статистика покрытия
        print(f"\n📊 Статистика покрытия ({example_sc}):")
        print(f"   Средний Days Cover: {sc_cover['days_cover'].mean():.1f} дней")
        print(f"   Медианный Days Cover: {sc_cover['days_cover'].median():.1f} дней")
        print(f"   Минимальный: {sc_cover['days_cover'].min():.1f} дней")
        print(f"   Максимальный: {sc_cover['days_cover'].max():.1f} дней")
        print(f"   OOS дней: {sc_cover['is_oos'].sum()} ({100*sc_cover['is_oos'].sum()/len(sc_cover):.1f}%)")
else:
    print("⚠️ Не удалось рассчитать покрытие")


## 4. Классификация спроса


In [ ]:
def classify_demand(sales_series):
    """Классификация спроса: stable, seasonal, intermittent"""
    if len(sales_series) == 0:
        return 'intermittent'
    
    sales_array = np.array(sales_series)
    
    # Процент нулевых дней
    zero_pct = np.sum(sales_array == 0) / len(sales_array)
    
    # Средний спрос (только ненулевые дни)
    non_zero = sales_array[sales_array > 0]
    if len(non_zero) == 0:
        return 'intermittent'
    
    avg_demand = np.mean(non_zero)
    
    # CV (коэффициент вариации)
    cv = np.std(non_zero) / avg_demand if avg_demand > 0 else np.inf
    
    # Средний интервал между продажами
    sale_days = np.where(sales_array > 0)[0]
    if len(sale_days) > 1:
        avg_interval = np.mean(np.diff(sale_days))
    else:
        avg_interval = len(sales_array)
    
    # Классификация
    if zero_pct > 0.5:  # Более 50% нулевых дней
        return 'intermittent'
    elif cv > 1.5:  # Высокая вариабельность
        return 'seasonal'
    else:
        return 'stable'

# Классификация спроса для каждого solo-code
print("🔍 Классификация спроса...")
demand_classification = []

for solo_code in all_solo_codes:
    sc_sales = total_sales[total_sales['solo_code'] == solo_code]['sales'].values
    
    if len(sc_sales) == 0:
        continue
    
    # Метрики
    zero_pct = np.sum(sc_sales == 0) / len(sc_sales)
    non_zero = sc_sales[sc_sales > 0]
    avg_demand = np.mean(non_zero) if len(non_zero) > 0 else 0
    cv = np.std(non_zero) / avg_demand if avg_demand > 0 else np.inf
    
    sale_days = np.where(sc_sales > 0)[0]
    if len(sale_days) > 1:
        avg_interval = np.mean(np.diff(sale_days))
    else:
        avg_interval = len(sc_sales)
    
    # Классификация
    demand_class = classify_demand(sc_sales)
    
    demand_classification.append({
        'solo_code': solo_code,
        'zero_pct': zero_pct,
        'avg_demand': avg_demand,
        'cv': cv,
        'avg_interval': avg_interval,
        'demand_class': demand_class
    })

demand_df = pd.DataFrame(demand_classification)

print(f"✅ Классификация завершена для {len(demand_df)} solo-code")
print("\n📊 Распределение классов:")
print(demand_df['demand_class'].value_counts())

print("\n📋 Детальная статистика:")
print(demand_df[['solo_code', 'zero_pct', 'avg_demand', 'cv', 'avg_interval', 'demand_class']].to_string(index=False))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Распределение классов
ax1 = axes[0]
demand_df['demand_class'].value_counts().plot(kind='bar', ax=ax1, color=['green', 'orange', 'red'])
ax1.set_title('Распределение классов спроса')
ax1.set_ylabel('Количество')
ax1.set_xlabel('Класс')
ax1.grid(True, alpha=0.3, axis='y')

# CV vs Zero %
ax2 = axes[1]
for cls in demand_df['demand_class'].unique():
    cls_data = demand_df[demand_df['demand_class'] == cls]
    ax2.scatter(cls_data['zero_pct'], cls_data['cv'], label=cls, alpha=0.7, s=100)
ax2.set_title('CV vs Процент нулевых дней')
ax2.set_xlabel('Процент нулевых дней')
ax2.set_ylabel('CV (коэффициент вариации)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Календарные признаки


In [ ]:
def create_calendar_features(dates):
    """Создание календарных признаков"""
    features = pd.DataFrame({'date': dates})
    
    # Базовые признаки
    features['day_of_week'] = features['date'].dt.dayofweek
    features['is_weekend'] = (features['day_of_week'] >= 5).astype(int)
    features['week_of_year'] = features['date'].dt.isocalendar().week
    features['month'] = features['date'].dt.month
    features['quarter'] = features['date'].dt.quarter
    features['year'] = features['date'].dt.year
    
    # Циклические признаки (sin/cos)
    features['week_sin'] = np.sin(2 * np.pi * features['week_of_year'] / 52)
    features['week_cos'] = np.cos(2 * np.pi * features['week_of_year'] / 52)
    features['month_sin'] = np.sin(2 * np.pi * features['month'] / 12)
    features['month_cos'] = np.cos(2 * np.pi * features['month'] / 12)
    
    # Праздники РФ с окнами
    features['is_new_year_period'] = 0
    features['is_feb_14_window'] = 0
    features['is_feb_23_window'] = 0
    features['is_mar_8_window'] = 0
    features['is_black_friday_period'] = 0
    
    for idx, date in enumerate(features['date']):
        year = date.year
        month = date.month
        day = date.day
        day_of_year = date.timetuple().tm_yday
        
        # Новый год: 15.12 - 10.01
        if (month == 12 and day >= 15) or (month == 1 and day <= 10):
            features.loc[idx, 'is_new_year_period'] = 1
        
        # 14 февраля ±7 дней
        if month == 2 and abs(day - 14) <= 7:
            features.loc[idx, 'is_feb_14_window'] = 1
        
        # 23 февраля ±7 дней
        if month == 2 and abs(day - 23) <= 7:
            features.loc[idx, 'is_feb_23_window'] = 1
        
        # 8 марта ±7 дней
        if month == 3 and abs(day - 8) <= 7:
            features.loc[idx, 'is_mar_8_window'] = 1
        
        # Черная пятница: последние 2 недели ноября
        if month == 11 and day >= 15:
            features.loc[idx, 'is_black_friday_period'] = 1
    
    return features

# Создание календарных признаков
print("📅 Создание календарных признаков...")
calendar_features = create_calendar_features(FULL_CALENDAR)

print(f"✅ Календарные признаки созданы: {len(calendar_features.columns)} признаков")
print(f"   Колонки: {list(calendar_features.columns)}")

# Проверка влияния праздников на продажи
if len(all_solo_codes) > 0:
    example_sc = all_solo_codes[0]
    sc_sales = total_sales[total_sales['solo_code'] == example_sc].copy()
    sc_sales = sc_sales.merge(calendar_features, on='date', how='left')
    
    print(f"\n📊 Влияние праздников на продажи ({example_sc}):")
    for holiday in ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window', 
                    'is_mar_8_window', 'is_black_friday_period']:
        holiday_sales = sc_sales[sc_sales[holiday] == 1]['sales'].mean()
        normal_sales = sc_sales[sc_sales[holiday] == 0]['sales'].mean()
        if normal_sales > 0:
            impact = (holiday_sales / normal_sales - 1) * 100
            print(f"   {holiday}: {holiday_sales:.1f} vs {normal_sales:.1f} ({impact:+.1f}%)")
    
    # Визуализация
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Продажи с выделением праздников
    ax1 = axes[0]
    ax1.plot(sc_sales['date'], sc_sales['sales'], alpha=0.5, label='Продажи')
    
    for holiday, color in [('is_new_year_period', 'red'), 
                          ('is_black_friday_period', 'orange'),
                          ('is_mar_8_window', 'pink')]:
        holiday_dates = sc_sales[sc_sales[holiday] == 1]['date']
        if len(holiday_dates) > 0:
            ax1.scatter(holiday_dates, sc_sales[sc_sales[holiday] == 1]['sales'],
                       label=holiday, alpha=0.7, s=50, color=color)
    
    ax1.set_title(f'Продажи с выделением праздников: {example_sc}')
    ax1.set_ylabel('Продажи')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Средние продажи по дням недели
    ax2 = axes[1]
    weekday_sales = sc_sales.groupby('day_of_week')['sales'].mean()
    weekday_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
    ax2.bar(range(7), weekday_sales.values, alpha=0.7)
    ax2.set_xticks(range(7))
    ax2.set_xticklabels(weekday_names)
    ax2.set_title(f'Средние продажи по дням недели: {example_sc}')
    ax2.set_ylabel('Средние продажи')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()


## 6. Бизнес-ограничения

### 6.1. Withdraw (вывод из продажи)


In [ ]:
# Загрузка справочника withdraw
withdraw_df = data.get('withdraw', pd.DataFrame())
if len(withdraw_df) > 0:
    withdraw_df.columns = [col.lower().replace('-', '_') for col in withdraw_df.columns]
    if 'solo_code' not in withdraw_df.columns and 'solo-code' in withdraw_df.columns:
        withdraw_df['solo_code'] = withdraw_df['solo-code']
    
    print(f"⚠️ Withdraw: {len(withdraw_df)} записей")
    print(f"   Solo-codes: {withdraw_df['solo_code'].unique().tolist()}")
else:
    print("✅ Withdraw: нет записей")

# Функция для обработки withdraw
def apply_withdraw_constraint(sales_series, solo_code, withdraw_df):
    """Применение ограничения withdraw: не прогнозировать новый спрос, распределить остатки"""
    if len(withdraw_df) == 0:
        return sales_series
    
    if solo_code in withdraw_df['solo_code'].values:
        # Для withdraw: прогноз = 0, остатки будут распределены равномерно до нуля
        return np.zeros_like(sales_series)
    
    return sales_series


### 6.2. Defecture (дефектура)


In [ ]:
# Загрузка справочника defecture
defecture_df = data.get('defecture', pd.DataFrame())
if len(defecture_df) > 0:
    defecture_df.columns = [col.lower().replace('-', '_') for col in defecture_df.columns]
    if 'solo_code' not in defecture_df.columns and 'solo-code' in defecture_df.columns:
        defecture_df['solo_code'] = defecture_df['solo-code']
    
    # Преобразование даты окончания дефектуры
    date_col = [col for col in defecture_df.columns if 'дата' in col.lower() or 'date' in col.lower()]
    if len(date_col) > 0:
        defecture_df['defecture_end_date'] = pd.to_datetime(defecture_df[date_col[0]])
    
    print(f"⚠️ Defecture: {len(defecture_df)} записей")
    print(defecture_df.to_string(index=False))
else:
    print("✅ Defecture: нет записей")

# Функция для обработки defecture
def apply_defecture_constraint(sales_series, dates, solo_code, defecture_df):
    """Применение ограничения defecture: продажи = 0 в период дефектуры"""
    if len(defecture_df) == 0:
        return sales_series
    
    defecture_records = defecture_df[defecture_df['solo_code'] == solo_code]
    if len(defecture_records) == 0:
        return sales_series
    
    result = sales_series.copy()
    
    for _, record in defecture_records.iterrows():
        if 'defecture_end_date' in record:
            end_date = pd.to_datetime(record['defecture_end_date'])
            # Продажи = 0 до даты окончания дефектуры
            mask = dates <= end_date
            result[mask] = 0
    
    return result


## 7. Модели прогнозирования продаж

### 7.1. Baseline модели


In [ ]:
class BaselineModels:
    """Базовые модели прогнозирования"""
    
    @staticmethod
    def naive(series, horizon=1):
        """Naive: последнее значение"""
        if len(series) == 0:
            return np.zeros(horizon)
        return np.full(horizon, series[-1])
    
    @staticmethod
    def moving_average(series, window=7, horizon=1):
        """Moving Average"""
        if len(series) < window:
            return BaselineModels.naive(series, horizon)
        return np.full(horizon, np.mean(series[-window:]))
    
    @staticmethod
    def seasonal_naive(series, season_length=7, horizon=1):
        """Seasonal Naive: значение сезона назад"""
        if len(series) < season_length:
            return BaselineModels.naive(series, horizon)
        return np.tile(series[-season_length:], (horizon // season_length + 1))[:horizon]
    
    @staticmethod
    def croston(series, horizon=1, alpha=0.1):
        """Croston для intermittent demand"""
        if len(series) == 0:
            return np.zeros(horizon)
        
        non_zero = series[series > 0]
        if len(non_zero) == 0:
            return np.zeros(horizon)
        
        # Простое экспоненциальное сглаживание для ненулевых значений
        if len(non_zero) == 1:
            forecast = non_zero[0]
        else:
            forecast = non_zero[0]
            for val in non_zero[1:]:
                forecast = alpha * val + (1 - alpha) * forecast
        
        # Учет частоты продаж
        intervals = []
        last_idx = -1
        for i, val in enumerate(series):
            if val > 0:
                if last_idx >= 0:
                    intervals.append(i - last_idx)
                last_idx = i
        
        avg_interval = np.mean(intervals) if len(intervals) > 0 else len(series)
        daily_rate = forecast / avg_interval if avg_interval > 0 else 0
        
        return np.full(horizon, daily_rate)
    
    @staticmethod
    def sba(series, horizon=1, alpha=0.1):
        """SBA (Syntetos-Boylan Approximation)"""
        croston_forecast = BaselineModels.croston(series, horizon, alpha)
        return croston_forecast * 0.5  # SBA корректировка
    
    @staticmethod
    def tsb(series, horizon=1, alpha=0.1, beta=0.1):
        """TSB (Teunter-Syntetos-Babai)"""
        if len(series) == 0:
            return np.zeros(horizon)
        
        # Вероятность продажи
        prob_sale = np.sum(series > 0) / len(series) if len(series) > 0 else 0
        
        # Размер продажи (если была)
        non_zero = series[series > 0]
        if len(non_zero) == 0:
            return np.zeros(horizon)
        
        if len(non_zero) == 1:
            size_forecast = non_zero[0]
        else:
            size_forecast = non_zero[0]
            for val in non_zero[1:]:
                size_forecast = beta * val + (1 - beta) * size_forecast
        
        # Обновление вероятности
        if len(series) > 0:
            last_was_sale = 1 if series[-1] > 0 else 0
            prob_sale = alpha * last_was_sale + (1 - alpha) * prob_sale
        
        forecast = prob_sale * size_forecast
        return np.full(horizon, forecast)

print("✅ Baseline модели определены")


### 7.2. Продвинутые модели


In [ ]:
class AdvancedModels:
    """Продвинутые модели прогнозирования"""
    
    @staticmethod
    def linear_regression(train_data, train_features, test_features, horizon=1):
        """Linear Regression с лагами и календарными признаками"""
        try:
            if len(train_data) < 10:
                return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)
            
            # Создание лагов
            max_lag = min(7, len(train_data) - 1)
            X_train = []
            y_train = train_data[max_lag:]
            
            for i in range(max_lag, len(train_data)):
                features = list(train_features.iloc[i, 1:].values)  # Пропускаем date
                lags = list(train_data[i-max_lag:i])
                X_train.append(features + lags)
            
            X_train = np.array(X_train)
            
            # Подготовка тестовых данных
            X_test = []
            last_features = list(test_features.iloc[0, 1:].values)
            last_values = list(train_data[-max_lag:])
            
            for h in range(horizon):
                X_test.append(last_features + last_values)
                # Обновляем лаги (используем прогноз)
                if h < horizon - 1:
                    last_values = last_values[1:] + [0]  # Заглушка
            
            X_test = np.array(X_test)
            
            # Обучение модели
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Прогноз
            forecast = model.predict(X_test)
            forecast = np.maximum(forecast, 0)  # Неотрицательные значения
            
            return forecast
        except Exception as e:
            print(f"⚠️ Ошибка Linear Regression: {e}")
            return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)
    
    @staticmethod
    def arima(train_data, horizon=1, order=(1, 1, 1)):
        """ARIMA"""
        try:
            if len(train_data) < 10:
                return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)
            
            # Автоматический подбор порядка для коротких рядов
            if len(train_data) < 50:
                order = (1, 0, 1)
            
            model = ARIMA(train_data, order=order)
            fitted = model.fit()
            forecast = fitted.forecast(steps=horizon)
            
            return np.maximum(forecast, 0)
        except Exception as e:
            print(f"⚠️ Ошибка ARIMA: {e}")
            return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)
    
    @staticmethod
    def sarima(train_data, horizon=1, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7)):
        """SARIMA"""
        try:
            if len(train_data) < 20:
                return AdvancedModels.arima(train_data, horizon)
            
            model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
            fitted = model.fit(disp=False)
            forecast = fitted.forecast(steps=horizon)
            
            return np.maximum(forecast, 0)
        except Exception as e:
            print(f"⚠️ Ошибка SARIMA: {e}")
            return AdvancedModels.arima(train_data, horizon)
    
    @staticmethod
    def sarimax(train_data, exog_train, exog_test, horizon=1):
        """SARIMAX с экзогенными переменными"""
        try:
            if len(train_data) < 20:
                return AdvancedModels.arima(train_data, horizon)
            
            # Подготовка экзогенных переменных
            exog_train_vals = exog_train.iloc[:, 1:].values  # Пропускаем date
            exog_test_vals = exog_test.iloc[:, 1:].values
            
            model = SARIMAX(train_data, exog=exog_train_vals, 
                          order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
            fitted = model.fit(disp=False)
            forecast = fitted.forecast(steps=horizon, exog=exog_test_vals)
            
            return np.maximum(forecast, 0)
        except Exception as e:
            print(f"⚠️ Ошибка SARIMAX: {e}")
            return AdvancedModels.sarima(train_data, horizon)
    
    @staticmethod
    def prophet(train_data, dates_train, dates_test, horizon=1):
        """Prophet с праздниками"""
        try:
            if len(train_data) < 30:
                return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)
            
            # Подготовка данных для Prophet
            df = pd.DataFrame({
                'ds': dates_train,
                'y': train_data
            })
            
            # Праздники
            holidays = pd.DataFrame({
                'holiday': ['new_year', 'feb_14', 'feb_23', 'mar_8', 'black_friday'],
                'ds': [
                    pd.to_datetime('2024-12-31'),
                    pd.to_datetime('2024-02-14'),
                    pd.to_datetime('2024-02-23'),
                    pd.to_datetime('2024-03-08'),
                    pd.to_datetime('2024-11-29')
                ],
                'lower_window': [-15, -7, -7, -7, -14],
                'upper_window': [10, 7, 7, 7, 0]
            })
            
            model = Prophet(holidays=holidays, yearly_seasonality=True, weekly_seasonality=True)
            model.fit(df)
            
            # Прогноз
            future = model.make_future_dataframe(periods=horizon)
            forecast = model.predict(future)
            
            return np.maximum(forecast['yhat'].tail(horizon).values, 0)
        except Exception as e:
            print(f"⚠️ Ошибка Prophet: {e}")
            return np.full(horizon, np.mean(train_data) if len(train_data) > 0 else 0)

print("✅ Продвинутые модели определены")


## 8. Валидация и выбор модели


In [ ]:
def calculate_mape(y_true, y_pred):
    """Robust MAPE"""
    mask = y_true != 0
    if mask.sum() == 0:
        return np.inf
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def rolling_validation(series, dates, calendar_features, demand_class, n_splits=5, test_size=30):
    """Rolling window validation"""
    if len(series) < test_size * 2:
        return {}
    
    min_train_size = max(30, len(series) - test_size * n_splits)
    results = {}
    
    # Список моделей для тестирования
    models_to_test = {
        'naive': lambda s: BaselineModels.naive(s, test_size),
        'moving_average': lambda s: BaselineModels.moving_average(s, 7, test_size),
        'seasonal_naive': lambda s: BaselineModels.seasonal_naive(s, 7, test_size),
    }
    
    # Добавляем модели в зависимости от класса спроса
    if demand_class == 'intermittent':
        models_to_test.update({
            'croston': lambda s: BaselineModels.croston(s, test_size),
            'sba': lambda s: BaselineModels.sba(s, test_size),
            'tsb': lambda s: BaselineModels.tsb(s, test_size),
        })
    else:
        models_to_test.update({
            'arima': lambda s: AdvancedModels.arima(s, test_size),
            'sarima': lambda s: AdvancedModels.sarima(s, test_size),
        })
    
    # Если достаточно данных, добавляем продвинутые модели
    if len(series) >= 50:
        train_features = calendar_features.iloc[:len(series)-test_size]
        test_features = calendar_features.iloc[len(series)-test_size:len(series)]
        
        models_to_test['linear_regression'] = lambda s: AdvancedModels.linear_regression(
            s, train_features, test_features, test_size
        )
        models_to_test['sarimax'] = lambda s: AdvancedModels.sarimax(
            s, train_features, test_features, test_size
        )
    
    if len(series) >= 60:
        train_dates = dates[:len(series)-test_size]
        test_dates = dates[len(series)-test_size:len(series)]
        models_to_test['prophet'] = lambda s: AdvancedModels.prophet(
            s, train_dates, test_dates, test_size
        )
    
    # Валидация
    for model_name, model_func in models_to_test.items():
        try:
            # Последний фолд
            train_data = series[:-test_size]
            test_data = series[-test_size:]
            
            if len(train_data) < min_train_size:
                continue
            
            forecast = model_func(train_data)
            
            if len(forecast) != len(test_data):
                forecast = forecast[:len(test_data)]
            
            # Метрики
            mae = mean_absolute_error(test_data, forecast)
            rmse = np.sqrt(mean_squared_error(test_data, forecast))
            mape = calculate_mape(test_data, forecast)
            
            results[model_name] = {
                'mae': mae,
                'rmse': rmse,
                'mape': mape
            }
        except Exception as e:
            continue
    
    return results

# Валидация для каждого solo-code
print("🔍 Валидация моделей...")
validation_results = []

for solo_code in all_solo_codes[:5]:  # Ограничиваем для демонстрации
    sc_sales = total_sales[total_sales['solo_code'] == solo_code].copy()
    sc_sales = sc_sales.sort_values('date')
    
    if len(sc_sales) < 60:
        continue
    
    sales_series = sc_sales['sales'].values
    sales_dates = sc_sales['date'].values
    
    # Получаем класс спроса
    demand_class = demand_df[demand_df['solo_code'] == solo_code]['demand_class'].values[0]
    
    # Валидация
    results = rolling_validation(sales_series, sales_dates, calendar_features, demand_class)
    
    if len(results) > 0:
        # Выбор лучшей модели по MAPE
        best_model = min(results.items(), key=lambda x: x[1]['mape'])
        
        validation_results.append({
            'solo_code': solo_code,
            'best_model': best_model[0],
            'mape': best_model[1]['mape'],
            'mae': best_model[1]['mae'],
            'rmse': best_model[1]['rmse'],
            'all_results': results
        })

if len(validation_results) > 0:
    validation_df = pd.DataFrame(validation_results)
else:
    validation_df = pd.DataFrame()

if len(validation_df) > 0:
    print(f"✅ Валидация завершена для {len(validation_df)} solo-code")
    print("\n📊 Лучшие модели:")
    print(validation_df[['solo_code', 'best_model', 'mape', 'mae', 'rmse']].to_string(index=False))
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Распределение лучших моделей
    ax1 = axes[0]
    validation_df['best_model'].value_counts().plot(kind='bar', ax=ax1)
    ax1.set_title('Распределение лучших моделей')
    ax1.set_ylabel('Количество')
    ax1.set_xlabel('Модель')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # MAPE по моделям
    ax2 = axes[1]
    validation_df.boxplot(column='mape', by='best_model', ax=ax2)
    ax2.set_title('MAPE по моделям')
    ax2.set_xlabel('Модель')
    ax2.set_ylabel('MAPE (%)')
    plt.suptitle('')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Недостаточно данных для валидации")


## 9. Rolling-прогноз продаж (18 месяцев)


In [ ]:
# Rolling-прогноз на 18 месяцев (548 дней)
FORECAST_HORIZON_DAYS = 18 * 30  # 18 месяцев
FORECAST_START_DATE = END_DATE + timedelta(days=1)
FORECAST_DATES = pd.date_range(FORECAST_START_DATE, periods=FORECAST_HORIZON_DAYS, freq='D')

print(f"📅 Период прогноза: {FORECAST_START_DATE.date()} - {FORECAST_DATES[-1].date()}")
print(f"   Горизонт: {FORECAST_HORIZON_DAYS} дней")

# Создание календарных признаков для прогноза
forecast_calendar_features = create_calendar_features(FORECAST_DATES)

def generate_forecast(solo_code, sales_data, marketplace='Total'):
    """Генерация прогноза для solo-code"""
    sc_sales = sales_data[
        (sales_data['solo_code'] == solo_code) & 
        (sales_data['marketplace'] == marketplace)
    ].copy()
    
    if len(sc_sales) == 0:
        return None
    
    sc_sales = sc_sales.sort_values('date')
    sales_series = sc_sales['sales'].values
    sales_dates = sc_sales['date'].values
    
    # Получаем класс спроса и лучшую модель
    demand_class = demand_df[demand_df['solo_code'] == solo_code]['demand_class'].values[0] if len(demand_df[demand_df['solo_code'] == solo_code]) > 0 else 'stable'
    
    best_model_name = None
    if 'validation_df' in locals() and len(validation_df) > 0 and solo_code in validation_df['solo_code'].values:
        best_model_name = validation_df[validation_df['solo_code'] == solo_code]['best_model'].values[0]
    
    # Выбор модели для прогноза
    try:
        if best_model_name == 'naive':
            forecast = BaselineModels.naive(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'moving_average':
            forecast = BaselineModels.moving_average(sales_series, 7, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'seasonal_naive':
            forecast = BaselineModels.seasonal_naive(sales_series, 7, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'croston':
            forecast = BaselineModels.croston(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'sba':
            forecast = BaselineModels.sba(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'tsb':
            forecast = BaselineModels.tsb(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'arima':
            forecast = AdvancedModels.arima(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'sarima':
            forecast = AdvancedModels.sarima(sales_series, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'linear_regression':
            train_features = calendar_features.iloc[:len(sales_series)]
            forecast = AdvancedModels.linear_regression(sales_series, train_features, 
                                                       forecast_calendar_features, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'sarimax':
            train_features = calendar_features.iloc[:len(sales_series)]
            forecast = AdvancedModels.sarimax(sales_series, train_features, 
                                            forecast_calendar_features, FORECAST_HORIZON_DAYS)
        elif best_model_name == 'prophet':
            forecast = AdvancedModels.prophet(sales_series, sales_dates, FORECAST_DATES, FORECAST_HORIZON_DAYS)
        else:
            # Fallback: выбор по классу спроса
            if demand_class == 'intermittent':
                forecast = BaselineModels.tsb(sales_series, FORECAST_HORIZON_DAYS)
            elif demand_class == 'seasonal':
                forecast = AdvancedModels.sarima(sales_series, FORECAST_HORIZON_DAYS)
            else:
                forecast = BaselineModels.moving_average(sales_series, 7, FORECAST_HORIZON_DAYS)
    except Exception as e:
        print(f"⚠️ Ошибка прогноза для {solo_code}: {e}")
        forecast = BaselineModels.naive(sales_series, FORECAST_HORIZON_DAYS)
    
    # Применение бизнес-ограничений
    forecast = apply_withdraw_constraint(forecast, solo_code, withdraw_df)
    forecast = apply_defecture_constraint(forecast, FORECAST_DATES, solo_code, defecture_df)
    
    return forecast

# Генерация прогнозов
print("🔮 Генерация rolling-прогнозов...")
forecasts = {}

for solo_code in all_solo_codes:
    # Прогноз для Total
    forecast_total = generate_forecast(solo_code, sales_final, 'Total')
    if forecast_total is not None:
        forecasts[f"{solo_code}_Total"] = forecast_total
    
    # Прогноз для WB
    forecast_wb = generate_forecast(solo_code, sales_final, 'WB')
    if forecast_wb is not None:
        forecasts[f"{solo_code}_WB"] = forecast_wb
    
    # Прогноз для Ozon
    forecast_ozon = generate_forecast(solo_code, sales_final, 'Ozon')
    if forecast_ozon is not None:
        forecasts[f"{solo_code}_Ozon"] = forecast_ozon

print(f"✅ Прогнозы сгенерированы для {len(forecasts)} комбинаций")

# Визуализация прогнозов
if len(all_solo_codes) > 0:
    example_sc = all_solo_codes[0]
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    for idx, marketplace in enumerate(['Total', 'WB', 'Ozon']):
        key = f"{example_sc}_{marketplace}"
        if key in forecasts:
            ax = axes[idx]
            
            # Исторические данные
            hist_data = sales_final[
                (sales_final['solo_code'] == example_sc) & 
                (sales_final['marketplace'] == marketplace)
            ].sort_values('date')
            
            # Последние 90 дней истории
            hist_data = hist_data.tail(90)
            
            ax.plot(hist_data['date'], hist_data['sales'], 
                   label='Факт', alpha=0.7, linewidth=2)
            
            # Прогноз
            forecast_vals = forecasts[key]
            ax.plot(FORECAST_DATES, forecast_vals, 
                   label='Прогноз', alpha=0.7, linewidth=2, linestyle='--')
            
            # Соединение
            if len(hist_data) > 0:
                last_date = hist_data['date'].iloc[-1]
                last_value = hist_data['sales'].iloc[-1]
                ax.plot([last_date, FORECAST_DATES[0]], [last_value, forecast_vals[0]], 
                       'k--', alpha=0.3)
            
            ax.set_title(f'Прогноз продаж: {example_sc} - {marketplace}')
            ax.set_xlabel('Дата')
            ax.set_ylabel('Продажи')
            ax.legend()
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## 10. Анализ исторических отгрузок и Lead Time


In [ ]:
def analyze_shipments_and_leadtime(shipments_df, stocks_wb, stocks_ozon):
    """Анализ отгрузок и восстановление lead time"""
    if shipments_df is None or len(shipments_df) == 0:
        return None
    
    results = []
    
    for solo_code in all_solo_codes:
        sc_shipments = shipments_df[shipments_df['solo_code'] == solo_code].copy()
        sc_shipments = sc_shipments[sc_shipments['shipment'] > 0].sort_values('date')
        
        if len(sc_shipments) == 0:
            continue
        
        # Анализ частоты и размера партий
        shipment_dates = sc_shipments['date'].values
        shipment_quantities = sc_shipments['shipment'].values
        
        if len(shipment_dates) > 1:
            intervals = np.diff(shipment_dates).astype('timedelta64[D]').astype(int)
            avg_interval = np.mean(intervals)
            median_interval = np.median(intervals)
        else:
            avg_interval = 30
            median_interval = 30
        
        avg_batch_size = np.mean(shipment_quantities)
        std_batch_size = np.std(shipment_quantities)
        
        # Восстановление lead time (от даты отгрузки до роста остатков)
        lead_times = []
        
        if stocks_wb is not None:
            sc_stocks_wb = stocks_wb[stocks_wb['solo_code'] == solo_code].sort_values('date')
            
            for ship_date, ship_qty in zip(shipment_dates, shipment_quantities):
                # Ищем рост остатков после отгрузки
                stocks_after = sc_stocks_wb[sc_stocks_wb['date'] > ship_date].head(10)
                
                if len(stocks_after) > 0:
                    # Проверяем рост остатков
                    base_stock = sc_stocks_wb[sc_stocks_wb['date'] <= ship_date]
                    if len(base_stock) > 0:
                        base_stock_val = base_stock['stock'].iloc[-1]
                        
                        for idx, row in stocks_after.iterrows():
                            if row['stock'] > base_stock_val * 1.1:  # Рост на 10%
                                lead_time = (row['date'] - ship_date).days
                                if 0 < lead_time <= 30:  # Разумный диапазон
                                    lead_times.append(lead_time)
                                break
        
        if len(lead_times) == 0:
            # Если не удалось восстановить, используем средний интервал между отгрузками
            lead_times = [int(avg_interval)]
        
        avg_lead_time = np.mean(lead_times)
        p90_lead_time = np.percentile(lead_times, 90) if len(lead_times) > 0 else avg_lead_time
        
        results.append({
            'solo_code': solo_code,
            'avg_interval': avg_interval,
            'median_interval': median_interval,
            'avg_batch_size': avg_batch_size,
            'std_batch_size': std_batch_size,
            'avg_lead_time': avg_lead_time,
            'p90_lead_time': p90_lead_time,
            'n_shipments': len(sc_shipments)
        })
    
    return pd.DataFrame(results)

# Анализ отгрузок и lead time
print("📦 Анализ исторических отгрузок и lead time...")
shipment_analysis = analyze_shipments_and_leadtime(shipments_clean, wb_stocks_clean, ozon_stocks_clean)

if shipment_analysis is not None and len(shipment_analysis) > 0:
    print(f"✅ Анализ завершен для {len(shipment_analysis)} solo-code")
    print("\n📊 Статистика отгрузок и lead time:")
    print(shipment_analysis.to_string(index=False))
    
    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Средний интервал между отгрузками
    ax1 = axes[0, 0]
    ax1.bar(range(len(shipment_analysis)), shipment_analysis['avg_interval'], alpha=0.7)
    ax1.set_title('Средний интервал между отгрузками')
    ax1.set_xlabel('Solo-code')
    ax1.set_ylabel('Дней')
    ax1.set_xticks(range(len(shipment_analysis)))
    ax1.set_xticklabels(shipment_analysis['solo_code'], rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Средний размер партии
    ax2 = axes[0, 1]
    ax2.bar(range(len(shipment_analysis)), shipment_analysis['avg_batch_size'], alpha=0.7)
    ax2.set_title('Средний размер партии')
    ax2.set_xlabel('Solo-code')
    ax2.set_ylabel('Количество')
    ax2.set_xticks(range(len(shipment_analysis)))
    ax2.set_xticklabels(shipment_analysis['solo_code'], rotation=45)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Lead time
    ax3 = axes[1, 0]
    ax3.bar(range(len(shipment_analysis)), shipment_analysis['avg_lead_time'], 
           alpha=0.7, label='Средний')
    ax3.bar(range(len(shipment_analysis)), shipment_analysis['p90_lead_time'], 
           alpha=0.5, label='P90')
    ax3.set_title('Lead Time')
    ax3.set_xlabel('Solo-code')
    ax3.set_ylabel('Дней')
    ax3.set_xticks(range(len(shipment_analysis)))
    ax3.set_xticklabels(shipment_analysis['solo_code'], rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Вариабельность размера партии
    ax4 = axes[1, 1]
    ax4.scatter(shipment_analysis['avg_batch_size'], shipment_analysis['std_batch_size'], 
               alpha=0.7, s=100)
    ax4.set_title('Вариабельность размера партии')
    ax4.set_xlabel('Средний размер партии')
    ax4.set_ylabel('Ст. отклонение')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Недостаточно данных для анализа")
    # Создаем заглушку
    shipment_analysis = pd.DataFrame({
        'solo_code': all_solo_codes,
        'avg_interval': [30] * len(all_solo_codes),
        'median_interval': [30] * len(all_solo_codes),
        'avg_batch_size': [200] * len(all_solo_codes),
        'std_batch_size': [50] * len(all_solo_codes),
        'avg_lead_time': [7] * len(all_solo_codes),
        'p90_lead_time': [14] * len(all_solo_codes),
        'n_shipments': [10] * len(all_solo_codes)
    })


## 11. Расчет страхового запаса


In [ ]:
# Расчет страхового запаса
# Safety Stock = Z * σ * sqrt(lead_time)
# Service Level = 95% -> Z = 1.65

SERVICE_LEVEL = 0.95
Z_SCORE = 1.65  # Для 95% service level

def calculate_safety_stock(sales_series, lead_time):
    """Расчет страхового запаса"""
    if len(sales_series) == 0:
        return 0
    
    # Стандартное отклонение дневного спроса
    daily_demand_std = np.std(sales_series)
    
    # Страховой запас
    safety_stock = Z_SCORE * daily_demand_std * np.sqrt(lead_time)
    
    return max(0, safety_stock)

print("📊 Расчет страхового запаса...")
safety_stocks = []

for solo_code in all_solo_codes:
    sc_sales = total_sales[total_sales['solo_code'] == solo_code]['sales'].values
    
    if len(sc_sales) == 0:
        continue
    
    # Получаем lead time
    if shipment_analysis is not None and solo_code in shipment_analysis['solo_code'].values:
        lead_time = shipment_analysis[shipment_analysis['solo_code'] == solo_code]['avg_lead_time'].values[0]
    else:
        lead_time = 7  # По умолчанию
    
    safety_stock = calculate_safety_stock(sc_sales, lead_time)
    
    # Дополнительная статистика
    avg_daily_demand = np.mean(sc_sales)
    daily_demand_std = np.std(sc_sales)
    
    safety_stocks.append({
        'solo_code': solo_code,
        'avg_daily_demand': avg_daily_demand,
        'daily_demand_std': daily_demand_std,
        'lead_time': lead_time,
        'safety_stock': safety_stock,
        'days_cover_safety': safety_stock / avg_daily_demand if avg_daily_demand > 0 else 0
    })

safety_stock_df = pd.DataFrame(safety_stocks)

print(f"✅ Страховой запас рассчитан для {len(safety_stock_df)} solo-code")
print("\n📋 Таблица страхового запаса:")
print(safety_stock_df.to_string(index=False))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Страховой запас по solo-code
ax1 = axes[0]
ax1.bar(range(len(safety_stock_df)), safety_stock_df['safety_stock'], alpha=0.7)
ax1.set_title('Страховой запас по solo-code')
ax1.set_xlabel('Solo-code')
ax1.set_ylabel('Страховой запас')
ax1.set_xticks(range(len(safety_stock_df)))
ax1.set_xticklabels(safety_stock_df['solo_code'], rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Дни покрытия страховым запасом
ax2 = axes[1]
ax2.bar(range(len(safety_stock_df)), safety_stock_df['days_cover_safety'], alpha=0.7)
ax2.set_title('Дни покрытия страховым запасом')
ax2.set_xlabel('Solo-code')
ax2.set_ylabel('Дней')
ax2.set_xticks(range(len(safety_stock_df)))
ax2.set_xticklabels(safety_stock_df['solo_code'], rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 12. Планирование отгрузок


In [ ]:
# Загрузка справочника count_box
count_box_df = data.get('count_box', pd.DataFrame())
if len(count_box_df) > 0:
    count_box_df.columns = [col.lower().replace('-', '_') for col in count_box_df.columns]
    if 'solo_code' not in count_box_df.columns and 'solo-code' in count_box_df.columns:
        count_box_df['solo_code'] = count_box_df['solo-code']
    count_col = [col for col in count_box_df.columns if 'кол' in col.lower() or 'count' in col.lower()][0]
    count_box_df['count_per_box'] = count_box_df[count_col]
    
    print(f"📦 Count box: {len(count_box_df)} записей")
else:
    print("⚠️ Count box не найден, используется значение по умолчанию: 12")
    count_box_df = pd.DataFrame({
        'solo_code': all_solo_codes,
        'count_per_box': [12] * len(all_solo_codes)
    })

def plan_shipments(solo_code, forecast_dates, forecast_values, current_stocks_wb, current_stocks_ozon, 
                  current_stocks_our, safety_stock, avg_batch_size, count_per_box, lead_time):
    """Планирование отгрузок"""
    shipment_plan = []
    
    # Текущие остатки (последние известные значения)
    stock_wb = current_stocks_wb if current_stocks_wb is not None else 0
    stock_ozon = current_stocks_ozon if current_stocks_ozon is not None else 0
    stock_our = current_stocks_our if current_stocks_our is not None else 0
    
    # Целевой уровень запаса (средний спрос * покрытие + страховой запас)
    avg_daily_demand = np.mean(forecast_values) if len(forecast_values) > 0 else 0
    target_coverage_days = 30  # Целевое покрытие 30 дней
    target_stock = avg_daily_demand * target_coverage_days + safety_stock
    
    # Планирование на каждый месяц
    monthly_forecasts = []
    monthly_dates = []
    
    current_month = forecast_dates[0].month
    month_forecast = 0
    month_start_date = forecast_dates[0]
    
    for i, (date, forecast) in enumerate(zip(forecast_dates, forecast_values)):
        if date.month != current_month or i == len(forecast_dates) - 1:
            if i == len(forecast_dates) - 1:
                month_forecast += forecast
            
            monthly_forecasts.append(month_forecast)
            monthly_dates.append(month_start_date)
            
            if i < len(forecast_dates) - 1:
                current_month = date.month
                month_forecast = forecast
                month_start_date = date
        else:
            month_forecast += forecast
    
    # Определение дат отгрузок
    for month_idx, (month_date, month_forecast) in enumerate(zip(monthly_dates, monthly_forecasts)):
        # Прогнозируемые остатки в конце месяца
        projected_stock = stock_wb + stock_ozon - month_forecast
        
        # Чистая потребность
        net_requirement = max(0, target_stock - projected_stock)
        
        if net_requirement > 0:
            # Округление до короба
            boxes_needed = int(np.ceil(net_requirement / count_per_box))
            packages_needed = boxes_needed * count_per_box
            
            # Рекомендуемая дата отгрузки (с учетом lead time)
            shipment_date = month_date - timedelta(days=int(lead_time))
            
            shipment_plan.append({
                'date': shipment_date,
                'solo_code': solo_code,
                'forecast_monthly': month_forecast,
                'current_stock_wb': stock_wb,
                'current_stock_ozon': stock_ozon,
                'current_stock_our': stock_our,
                'safety_stock': safety_stock,
                'target_stock': target_stock,
                'net_requirement': net_requirement,
                'shipment_packages': packages_needed,
                'shipment_boxes': boxes_needed
            })
            
            # Обновление остатков после отгрузки
            stock_our -= packages_needed
            stock_wb += packages_needed  # Упрощенная логика
    
    return pd.DataFrame(shipment_plan)

# Планирование отгрузок для всех solo-code
print("📅 Планирование отгрузок...")
shipment_plans = []

for solo_code in all_solo_codes:
    # Прогноз
    forecast_key = f"{solo_code}_Total"
    if forecast_key not in forecasts:
        continue
    
    forecast_vals = forecasts[forecast_key]
    
    # Текущие остатки (последние известные значения)
    current_stock_wb = None
    if wb_stocks_clean is not None:
        sc_stocks_wb = wb_stocks_clean[wb_stocks_clean['solo_code'] == solo_code]
        if len(sc_stocks_wb) > 0:
            current_stock_wb = sc_stocks_wb['stock'].iloc[-1]
    
    current_stock_ozon = None
    if ozon_stocks_clean is not None:
        sc_stocks_ozon = ozon_stocks_clean[ozon_stocks_clean['solo_code'] == solo_code]
        if len(sc_stocks_ozon) > 0:
            current_stock_ozon = sc_stocks_ozon['stock'].iloc[-1]
    
    current_stock_our = None
    if our_stocks_clean is not None:
        sc_stocks_our = our_stocks_clean[our_stocks_clean['solo_code'] == solo_code]
        if len(sc_stocks_our) > 0:
            current_stock_our = sc_stocks_our['stock'].iloc[-1]
    
    # Страховой запас
    safety_stock = 0
    if solo_code in safety_stock_df['solo_code'].values:
        safety_stock = safety_stock_df[safety_stock_df['solo_code'] == solo_code]['safety_stock'].values[0]
    
    # Параметры отгрузки
    avg_batch_size = 200
    if shipment_analysis is not None and solo_code in shipment_analysis['solo_code'].values:
        avg_batch_size = shipment_analysis[shipment_analysis['solo_code'] == solo_code]['avg_batch_size'].values[0]
    
    count_per_box = 12
    if solo_code in count_box_df['solo_code'].values:
        count_per_box = count_box_df[count_box_df['solo_code'] == solo_code]['count_per_box'].values[0]
    
    lead_time = 7
    if shipment_analysis is not None and solo_code in shipment_analysis['solo_code'].values:
        lead_time = shipment_analysis[shipment_analysis['solo_code'] == solo_code]['avg_lead_time'].values[0]
    
    # Планирование
    plan = plan_shipments(solo_code, FORECAST_DATES, forecast_vals, 
                         current_stock_wb, current_stock_ozon, current_stock_our,
                         safety_stock, avg_batch_size, count_per_box, lead_time)
    
    if len(plan) > 0:
        shipment_plans.append(plan)

if len(shipment_plans) > 0:
    final_shipment_plan = pd.concat(shipment_plans, ignore_index=True)
    final_shipment_plan = final_shipment_plan.sort_values('date')
    
    print(f"✅ План отгрузок создан: {len(final_shipment_plan)} записей")
    print("\n📋 Финальная таблица планирования отгрузок:")
    print(final_shipment_plan.to_string(index=False))
    
    # Экспорт в CSV
    final_shipment_plan.to_csv('shipment_plan.csv', index=False, encoding='utf-8-sig')
    print("\n💾 План сохранен в shipment_plan.csv")
    
    # Визуализация
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # График отгрузок по датам
    ax1 = axes[0]
    for solo_code in final_shipment_plan['solo_code'].unique():
        sc_plan = final_shipment_plan[final_shipment_plan['solo_code'] == solo_code]
        ax1.bar(sc_plan['date'], sc_plan['shipment_packages'], 
               label=solo_code, alpha=0.7, width=5)
    ax1.set_title('Планируемые отгрузки по датам')
    ax1.set_xlabel('Дата')
    ax1.set_ylabel('Количество упаковок')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Суммарные отгрузки по solo-code
    ax2 = axes[1]
    total_by_sc = final_shipment_plan.groupby('solo_code')['shipment_packages'].sum()
    ax2.bar(range(len(total_by_sc)), total_by_sc.values, alpha=0.7)
    ax2.set_title('Суммарные отгрузки по solo-code')
    ax2.set_xlabel('Solo-code')
    ax2.set_ylabel('Количество упаковок')
    ax2.set_xticks(range(len(total_by_sc)))
    ax2.set_xticklabels(total_by_sc.index, rotation=45)
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Не удалось создать план отгрузок")
    final_shipment_plan = pd.DataFrame()  # Инициализация пустого DataFrame


## 13. Финальный Summary


In [ ]:
print("=" * 80)
print("📊 ФИНАЛЬНЫЙ SUMMARY")
print("=" * 80)

# 1. Summary по моделям
print("\n1️⃣ SUMMARY ПО МОДЕЛЯМ")
print("-" * 80)
if 'validation_df' in locals() and len(validation_df) > 0:
    print(f"✅ Валидация выполнена для {len(validation_df)} solo-code")
    print(f"\nРаспределение лучших моделей:")
    model_dist = validation_df['best_model'].value_counts()
    for model, count in model_dist.items():
        print(f"   {model}: {count} ({100*count/len(validation_df):.1f}%)")
    
    print(f"\nСредние метрики:")
    print(f"   MAPE: {validation_df['mape'].mean():.2f}%")
    print(f"   MAE: {validation_df['mae'].mean():.2f}")
    print(f"   RMSE: {validation_df['rmse'].mean():.2f}")
else:
    print("⚠️ Валидация не выполнена (недостаточно данных)")

# 2. Ошибки прогнозирования
print("\n2️⃣ ОШИБКИ ПРОГНОЗИРОВАНИЯ")
print("-" * 80)
if 'validation_df' in locals() and len(validation_df) > 0:
    print(f"   Средний MAPE: {validation_df['mape'].mean():.2f}%")
    print(f"   Медианный MAPE: {validation_df['mape'].median():.2f}%")
    print(f"   Минимальный MAPE: {validation_df['mape'].min():.2f}%")
    print(f"   Максимальный MAPE: {validation_df['mape'].max():.2f}%")
    
    worst_forecast = validation_df.loc[validation_df['mape'].idxmax()]
    best_forecast = validation_df.loc[validation_df['mape'].idxmin()]
    
    print(f"\n   Лучший прогноз: {best_forecast['solo_code']} (MAPE: {best_forecast['mape']:.2f}%)")
    print(f"   Худший прогноз: {worst_forecast['solo_code']} (MAPE: {worst_forecast['mape']:.2f}%)")

# 3. Риски
print("\n3️⃣ РИСКИ")
print("-" * 80)

# Риск по withdraw
if len(withdraw_df) > 0:
    print(f"⚠️ Withdraw: {len(withdraw_df)} solo-code выведены из продажи")
    print(f"   {list(withdraw_df['solo_code'].unique())}")

# Риск по defecture
if len(defecture_df) > 0:
    print(f"⚠️ Defecture: {len(defecture_df)} solo-code в дефектуре")
    for _, row in defecture_df.iterrows():
        print(f"   {row['solo_code']}: до {row.get('defecture_end_date', 'N/A')}")

# Риск по покрытию
if days_cover_df is not None:
    low_cover = days_cover_df[days_cover_df['days_cover'] < 7]
    if len(low_cover) > 0:
        print(f"⚠️ Низкое покрытие (<7 дней): {len(low_cover)} записей")
        low_cover_sc = low_cover.groupby('solo_code')['days_cover'].min()
        print(f"   Solo-codes с низким покрытием: {list(low_cover_sc.index[:5])}")

# Риск по страховому запасу
if len(safety_stock_df) > 0:
    high_safety = safety_stock_df[safety_stock_df['days_cover_safety'] > 30]
    if len(high_safety) > 0:
        print(f"⚠️ Высокий страховой запас (>30 дней): {len(high_safety)} solo-code")

# 4. Рекомендации по развитию
print("\n4️⃣ РЕКОМЕНДАЦИИ ПО РАЗВИТИЮ")
print("-" * 80)
print("""
📌 Улучшение качества данных:
   • Увеличить частоту обновления остатков (минимум ежедневно)
   • Добавить данные о промо-акциях и рекламных кампаниях
   • Включить информацию о конкурентах и рыночных трендах

📌 Улучшение моделей:
   • Добавить ансамбли моделей (voting, stacking)
   • Внедрить deep learning модели (LSTM, Transformer) для сложных паттернов
   • Добавить автоматический подбор гиперпараметров (Optuna, Hyperopt)

📌 Улучшение планирования:
   • Учесть ограничения склада и логистики
   • Добавить оптимизацию размера партии (EOQ модель)
   • Внедрить multi-echelon inventory optimization

📌 Автоматизация:
   • Настроить автоматический пересчет прогнозов (ежедневно/еженедельно)
   • Добавить алерты при отклонениях от плана
   • Создать дашборд для мониторинга KPI

📌 Интеграция:
   • Подключить API маркетплейсов для реального времени
   • Интегрировать с WMS/ERP системами
   • Добавить автоматическое создание заказов
""")

# 5. Статистика по прогнозам
print("\n5️⃣ СТАТИСТИКА ПО ПРОГНОЗАМ")
print("-" * 80)
print(f"   Период прогноза: {FORECAST_START_DATE.date()} - {FORECAST_DATES[-1].date()}")
print(f"   Горизонт: {FORECAST_HORIZON_DAYS} дней ({FORECAST_HORIZON_DAYS/30:.1f} месяцев)")
print(f"   Solo-code с прогнозами: {len([k for k in forecasts.keys() if '_Total' in k])}")

if 'final_shipment_plan' in locals() and len(final_shipment_plan) > 0:
    print(f"\n   Планируемые отгрузки:")
    print(f"   • Всего отгрузок: {len(final_shipment_plan)}")
    print(f"   • Суммарный объем: {final_shipment_plan['shipment_packages'].sum():.0f} упаковок")
    print(f"   • Суммарный объем: {final_shipment_plan['shipment_boxes'].sum():.0f} коробов")
    print(f"   • Средний размер отгрузки: {final_shipment_plan['shipment_packages'].mean():.0f} упаковок")

print("\n" + "=" * 80)
print("✅ АНАЛИЗ ЗАВЕРШЕН")
print("=" * 80)


# Система прогнозирования продаж и планирования отгрузок
## Wildberries & Ozon

**Цель:** Построить систему rolling-прогноза продаж на 18 месяцев с планированием отгрузок

**Период данных:** 26.03.2024 – 20.12.2025 (дневная частота)

**Идентификатор:** solo-code


In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Прогнозирование
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Библиотеки загружены")


## 1. Загрузка данных

Поддерживаем загрузку CSV и Excel файлов


In [ ]:
def load_data(file_path):
    """Загрузка данных (CSV или Excel)"""
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path, encoding='utf-8')
    elif file_path.endswith(('.xlsx', '.xls')):
        return pd.read_excel(file_path)
    else:
        raise ValueError(f"Неподдерживаемый формат файла: {file_path}")

# Загрузка данных продаж
try:
    wb_sales = load_data('wb_sales.csv')
    print(f"✅ WB продажи: {wb_sales.shape}")
except:
    try:
        wb_sales = load_data('wb_sales.xlsx')
        print(f"✅ WB продажи: {wb_sales.shape}")
    except:
        print("⚠️ Файл wb_sales не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        wb_sales = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Количество упак.': np.random.poisson(10, len(dates))
        })

try:
    ozon_sales = load_data('ozon_sales.csv')
    print(f"✅ Ozon продажи: {ozon_sales.shape}")
except:
    try:
        ozon_sales = load_data('ozon_sales.xlsx')
        print(f"✅ Ozon продажи: {ozon_sales.shape}")
    except:
        print("⚠️ Файл ozon_sales не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        ozon_sales = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Количество упак.': np.random.poisson(8, len(dates))
        })

# Загрузка остатков
try:
    wb_stocks = load_data('wb_stocks.csv')
    print(f"✅ WB остатки: {wb_stocks.shape}")
except:
    try:
        wb_stocks = load_data('wb_stocks.xlsx')
        print(f"✅ WB остатки: {wb_stocks.shape}")
    except:
        print("⚠️ Файл wb_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        wb_stocks = pd.DataFrame({
            'Дата': dates,
            'Склад': 'WB',
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(50, 200, len(dates))
        })

try:
    ozon_stocks = load_data('ozon_stocks.csv')
    print(f"✅ Ozon остатки: {ozon_stocks.shape}")
except:
    try:
        ozon_stocks = load_data('ozon_stocks.xlsx')
        print(f"✅ Ozon остатки: {ozon_stocks.shape}")
    except:
        print("⚠️ Файл ozon_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        ozon_stocks = pd.DataFrame({
            'Дата': dates,
            'Склад': 'Ozon',
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(40, 150, len(dates))
        })

try:
    our_stocks = load_data('our_stocks.csv')
    print(f"✅ Наши остатки: {our_stocks.shape}")
except:
    try:
        our_stocks = load_data('our_stocks.xlsx')
        print(f"✅ Наши остатки: {our_stocks.shape}")
    except:
        print("⚠️ Файл our_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        our_stocks = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(100, 300, len(dates))
        })

# Загрузка справочников
try:
    withdraw = load_data('withdraw.csv')
    print(f"✅ Товары на вывод: {withdraw.shape}")
except:
    try:
        withdraw = load_data('withdraw.xlsx')
        print(f"✅ Товары на вывод: {withdraw.shape}")
    except:
        print("⚠️ Файл withdraw не найден, создаем пустой")
        withdraw = pd.DataFrame(columns=['solo-code', 'SKU'])

try:
    defecture = load_data('defecture.csv')
    print(f"✅ Дефектура: {defecture.shape}")
except:
    try:
        defecture = load_data('defecture.xlsx')
        print(f"✅ Дефектура: {defecture.shape}")
    except:
        print("⚠️ Файл defecture не найден, создаем пустой")
        defecture = pd.DataFrame(columns=['solo-code', 'SKU', 'Дата окончания дефектуры'])

try:
    count_box = load_data('count_box.csv')
    print(f"✅ Количество в коробе: {count_box.shape}")
except:
    try:
        count_box = load_data('count_box.xlsx')
        print(f"✅ Количество в коробе: {count_box.shape}")
    except:
        print("⚠️ Файл count_box не найден, создаем пример")
        count_box = pd.DataFrame({
            'solo-code': ['SKU001'],
            'Кол-во': [12]
        })

print("\n📊 Обзор загруженных данных:")
print(f"WB продажи: {wb_sales.shape}")
print(f"Ozon продажи: {ozon_sales.shape}")
print(f"WB остатки: {wb_stocks.shape}")
print(f"Ozon остатки: {ozon_stocks.shape}")
print(f"Наши остатки: {our_stocks.shape}")


## 2. Подготовка и очистка данных

### 2.1. Приведение дат к datetime и создание полного календаря


In [ ]:
def prepare_dates(df, date_col='Дата'):
    """Приведение дат к datetime"""
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    return df

# Приведение дат
wb_sales = prepare_dates(wb_sales)
ozon_sales = prepare_dates(ozon_sales)
wb_stocks = prepare_dates(wb_stocks)
ozon_stocks = prepare_dates(ozon_stocks)
our_stocks = prepare_dates(our_stocks)

if 'Дата окончания дефектуры' in defecture.columns:
    defecture['Дата окончания дефектуры'] = pd.to_datetime(defecture['Дата окончания дефектуры'], errors='coerce')

# Определение общего периода
all_dates = []
for df in [wb_sales, ozon_sales, wb_stocks, ozon_stocks, our_stocks]:
    if 'Дата' in df.columns:
        all_dates.extend(df['Дата'].dropna().tolist())

min_date = min(all_dates) if all_dates else pd.Timestamp('2024-03-26')
max_date = max(all_dates) if all_dates else pd.Timestamp('2025-12-20')

print(f"📅 Период данных: {min_date.date()} - {max_date.date()}")
print(f"📅 Всего дней: {(max_date - min_date).days + 1}")

# Получение всех solo-code
all_solo_codes = set()
for df in [wb_sales, ozon_sales, wb_stocks, ozon_stocks, our_stocks]:
    if 'solo-code' in df.columns:
        all_solo_codes.update(df['solo-code'].dropna().unique())

all_solo_codes = sorted(list(all_solo_codes))
print(f"📦 Уникальных solo-code: {len(all_solo_codes)}")
print(f"📦 Примеры: {all_solo_codes[:5]}")


In [ ]:
def create_full_calendar(min_date, max_date, solo_codes):
    """Создание полного календаря для всех solo-code"""
    dates = pd.date_range(min_date, max_date, freq='D')
    calendar = []
    for solo_code in solo_codes:
        for date in dates:
            calendar.append({'Дата': date, 'solo-code': solo_code})
    return pd.DataFrame(calendar)

# Создание полного календаря
full_calendar = create_full_calendar(min_date, max_date, all_solo_codes)
print(f"📅 Полный календарь: {full_calendar.shape}")
print(f"📅 Пример календаря:")
print(full_calendar.head(10))


### 2.2. Обработка продаж: заполнение пропусков нулями


In [ ]:
def process_sales(df, calendar, channel_name):
    """Обработка продаж с заполнением пропусков"""
    # Агрегация по дате и solo-code
    sales_agg = df.groupby(['Дата', 'solo-code'])['Количество упак.'].sum().reset_index()
    
    # Объединение с календарем
    sales_full = calendar.merge(sales_agg, on=['Дата', 'solo-code'], how='left')
    sales_full['Количество упак.'] = sales_full['Количество упак.'].fillna(0)
    sales_full['Канал'] = channel_name
    
    return sales_full

# Обработка продаж
wb_sales_processed = process_sales(wb_sales, full_calendar, 'WB')
ozon_sales_processed = process_sales(ozon_sales, full_calendar, 'Ozon')

# Суммарные продажи
total_sales = wb_sales_processed.merge(
    ozon_sales_processed[['Дата', 'solo-code', 'Количество упак.']],
    on=['Дата', 'solo-code'],
    how='left',
    suffixes=('_wb', '_ozon')
)
total_sales['Количество упак.'] = (
    total_sales['Количество упак._wb'].fillna(0) + 
    total_sales['Количество упак._ozon'].fillna(0)
)
total_sales = total_sales[['Дата', 'solo-code', 'Количество упак.']].copy()
total_sales['Канал'] = 'Total'

print("✅ Продажи обработаны")
print(f"\n📊 Статистика по продажам:")
print(f"WB - пропусков: {(wb_sales_processed['Количество упак.'] == 0).sum() / len(wb_sales_processed) * 100:.1f}%")
print(f"Ozon - пропусков: {(ozon_sales_processed['Количество упак.'] == 0).sum() / len(ozon_sales_processed) * 100:.1f}%")
print(f"\n📊 Пример данных WB:")
print(wb_sales_processed.head(10))


### 2.3. Обработка остатков: forward/backward fill


In [ ]:
def process_stocks(df, calendar, stock_name):
    """Обработка остатков с forward/backward fill"""
    # Агрегация по дате и solo-code (если несколько складов - суммируем)
    if 'Склад' in df.columns:
        stocks_agg = df.groupby(['Дата', 'solo-code'])['Остаток'].sum().reset_index()
    else:
        stocks_agg = df.groupby(['Дата', 'solo-code'])['Остаток'].sum().reset_index()
    
    # Объединение с календарем
    stocks_full = calendar.merge(stocks_agg, on=['Дата', 'solo-code'], how='left')
    
    # Сортировка по solo-code и дате
    stocks_full = stocks_full.sort_values(['solo-code', 'Дата'])
    
    # Forward fill, затем backward fill для начала ряда
    stocks_full['Остаток'] = stocks_full.groupby('solo-code')['Остаток'].fillna(method='ffill')
    stocks_full['Остаток'] = stocks_full.groupby('solo-code')['Остаток'].fillna(method='bfill')
    stocks_full['Остаток'] = stocks_full['Остаток'].fillna(0)
    
    # Проверка на отрицательные значения
    stocks_full['Остаток'] = stocks_full['Остаток'].clip(lower=0)
    
    stocks_full['Тип_остатка'] = stock_name
    
    return stocks_full

# Обработка остатков
wb_stocks_processed = process_stocks(wb_stocks, full_calendar, 'WB')
ozon_stocks_processed = process_stocks(ozon_stocks, full_calendar, 'Ozon')
our_stocks_processed = process_stocks(our_stocks, full_calendar, 'Our')

print("✅ Остатки обработаны")
print(f"\n📊 Статистика по остаткам:")
print(f"WB остатки - среднее: {wb_stocks_processed['Остаток'].mean():.1f}")
print(f"Ozon остатки - среднее: {ozon_stocks_processed['Остаток'].mean():.1f}")
print(f"Наши остатки - среднее: {our_stocks_processed['Остаток'].mean():.1f}")
print(f"\n📊 Пример данных остатков:")
print(wb_stocks_processed.head(10))


### 2.4. Сводная статистика по данным


In [ ]:
print("=" * 80)
print("📊 СВОДНАЯ СТАТИСТИКА ПО ДАННЫМ")
print("=" * 80)

print(f"\n📦 Продажи:")
print(f"  - WB: {len(wb_sales_processed)} записей, {wb_sales_processed['solo-code'].nunique()} товаров")
print(f"  - Ozon: {len(ozon_sales_processed)} записей, {ozon_sales_processed['solo-code'].nunique()} товаров")
print(f"  - Total: {len(total_sales)} записей, {total_sales['solo-code'].nunique()} товаров")

print(f"\n🏬 Остатки:")
print(f"  - WB: {len(wb_stocks_processed)} записей")
print(f"  - Ozon: {len(ozon_stocks_processed)} записей")
print(f"  - Наши: {len(our_stocks_processed)} записей")

print(f"\n📈 Пропуски в продажах (нулевые дни):")
for solo_code in all_solo_codes[:5]:  # Показываем первые 5
    wb_data = wb_sales_processed[wb_sales_processed['solo-code'] == solo_code]
    ozon_data = ozon_sales_processed[ozon_sales_processed['solo-code'] == solo_code]
    total_data = total_sales[total_sales['solo-code'] == solo_code]
    
    wb_zeros = (wb_data['Количество упак.'] == 0).sum() / len(wb_data) * 100
    ozon_zeros = (ozon_data['Количество упак.'] == 0).sum() / len(ozon_data) * 100
    total_zeros = (total_data['Количество упак.'] == 0).sum() / len(total_data) * 100
    
    print(f"  {solo_code}: WB={wb_zeros:.1f}%, Ozon={ozon_zeros:.1f}%, Total={total_zeros:.1f}%")

print("\n✅ Подготовка данных завершена")


## 3. EDA и классификация спроса

Для каждого solo-code анализируем:
- График продаж
- Доля нулевых дней
- Средний спрос
- Коэффициент вариации (CV)


In [ ]:
def calculate_demand_metrics(sales_data, solo_code):
    """Расчет метрик спроса для товара"""
    data = sales_data[sales_data['solo-code'] == solo_code].copy()
    data = data.sort_values('Дата')
    
    demand = data['Количество упак.'].values
    
    metrics = {
        'solo-code': solo_code,
        'mean_demand': np.mean(demand),
        'std_demand': np.std(demand),
        'cv': np.std(demand) / (np.mean(demand) + 1e-6),  # Коэффициент вариации
        'zero_ratio': (demand == 0).sum() / len(demand),
        'total_demand': np.sum(demand),
        'max_demand': np.max(demand),
        'min_demand': np.min(demand)
    }
    
    return metrics

# Расчет метрик для всех товаров
demand_metrics = []
for solo_code in all_solo_codes:
    metrics = calculate_demand_metrics(total_sales, solo_code)
    demand_metrics.append(metrics)

demand_df = pd.DataFrame(demand_metrics)

print("📊 Метрики спроса по товарам:")
print(demand_df.head(10))
print(f"\n📊 Общая статистика:")
print(demand_df.describe())


In [ ]:
def classify_demand(zero_ratio, cv):
    """Классификация типа спроса"""
    if zero_ratio > 0.5:
        return 'intermittent'  # Прерывистый спрос (>50% нулей)
    elif cv > 1.0:
        return 'seasonal'  # Сезонный/нестабильный (высокая вариативность)
    else:
        return 'stable'  # Стабильный спрос

# Классификация товаров
demand_df['demand_type'] = demand_df.apply(
    lambda x: classify_demand(x['zero_ratio'], x['cv']), 
    axis=1
)

print("📊 Классификация спроса:")
print(demand_df['demand_type'].value_counts())
print(f"\n📊 Распределение по типам:")
print(demand_df.groupby('demand_type').agg({
    'mean_demand': 'mean',
    'cv': 'mean',
    'zero_ratio': 'mean'
}).round(2))


In [ ]:
# Визуализация для примера товара
example_solo_code = all_solo_codes[0]
example_data = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# График продаж
axes[0].plot(example_data['Дата'], example_data['Количество упак.'], linewidth=1.5)
axes[0].set_title(f'Продажи по дням: {example_solo_code}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Количество упаковок')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Распределение спроса
axes[1].hist(example_data['Количество упак.'], bins=30, edgecolor='black', alpha=0.7)
axes[1].set_title(f'Распределение спроса: {example_solo_code}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Количество упаковок')
axes[1].set_ylabel('Частота')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики для примера
example_metrics = demand_df[demand_df['solo-code'] == example_solo_code].iloc[0]
print(f"\n📊 Метрики для {example_solo_code}:")
print(f"  Тип спроса: {example_metrics['demand_type']}")
print(f"  Средний спрос: {example_metrics['mean_demand']:.2f}")
print(f"  CV: {example_metrics['cv']:.2f}")
print(f"  Доля нулей: {example_metrics['zero_ratio']:.2%}")


## 4. Генерация календарных признаков

Создаем признаки без внешних данных:
- Базовые признаки (день недели, месяц, квартал и т.д.)
- Циклические признаки (sin/cos)
- Праздники РФ с окнами влияния


In [ ]:
def create_calendar_features(df):
    """Создание календарных признаков"""
    df = df.copy()
    
    # Базовые признаки
    df['day_of_week'] = df['Дата'].dt.dayofweek  # 0=понедельник, 6=воскресенье
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['week_of_year'] = df['Дата'].dt.isocalendar().week
    df['month'] = df['Дата'].dt.month
    df['quarter'] = df['Дата'].dt.quarter
    df['year'] = df['Дата'].dt.year
    df['day_of_month'] = df['Дата'].dt.day
    df['day_of_year'] = df['Дата'].dt.dayofyear
    
    # Циклические признаки для месяца (12 месяцев)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Циклические признаки для недели (52 недели)
    df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)
    
    # Циклические признаки для дня недели (7 дней)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    return df

# Применяем к продажам
total_sales = create_calendar_features(total_sales)
wb_sales_processed = create_calendar_features(wb_sales_processed)
ozon_sales_processed = create_calendar_features(ozon_sales_processed)

print("✅ Базовые календарные признаки созданы")
print(f"\n📊 Пример признаков:")
print(total_sales[['Дата', 'day_of_week', 'is_weekend', 'month', 'quarter', 
                   'month_sin', 'month_cos']].head(10))


In [ ]:
def create_holiday_features(df):
    """Создание признаков праздников РФ с окнами влияния"""
    df = df.copy()
    
    # Новый год: 15.12 - 10.01
    df['is_new_year_period'] = 0
    df.loc[
        ((df['Дата'].dt.month == 12) & (df['Дата'].dt.day >= 15)) |
        ((df['Дата'].dt.month == 1) & (df['Дата'].dt.day <= 10)),
        'is_new_year_period'
    ] = 1
    
    # 14 февраля: ±7 дней
    df['is_feb_14_window'] = 0
    for year in df['Дата'].dt.year.unique():
        feb_14 = pd.Timestamp(f'{year}-02-14')
        mask = (df['Дата'] >= feb_14 - timedelta(days=7)) & (df['Дата'] <= feb_14 + timedelta(days=7))
        df.loc[mask, 'is_feb_14_window'] = 1
    
    # 23 февраля: ±7 дней
    df['is_feb_23_window'] = 0
    for year in df['Дата'].dt.year.unique():
        feb_23 = pd.Timestamp(f'{year}-02-23')
        mask = (df['Дата'] >= feb_23 - timedelta(days=7)) & (df['Дата'] <= feb_23 + timedelta(days=7))
        df.loc[mask, 'is_feb_23_window'] = 1
    
    # 8 марта: ±7 дней
    df['is_mar_8_window'] = 0
    for year in df['Дата'].dt.year.unique():
        mar_8 = pd.Timestamp(f'{year}-03-08')
        mask = (df['Дата'] >= mar_8 - timedelta(days=7)) & (df['Дата'] <= mar_8 + timedelta(days=7))
        df.loc[mask, 'is_mar_8_window'] = 1
    
    # 1 сентября: ±7 дней
    df['is_sep_1_window'] = 0
    for year in df['Дата'].dt.year.unique():
        sep_1 = pd.Timestamp(f'{year}-09-01')
        mask = (df['Дата'] >= sep_1 - timedelta(days=7)) & (df['Дата'] <= sep_1 + timedelta(days=7))
        df.loc[mask, 'is_sep_1_window'] = 1
    
    # Черная пятница: последние 2 недели ноября
    df['is_black_friday_period'] = 0
    df.loc[
        (df['Дата'].dt.month == 11) & (df['Дата'].dt.day >= 15),
        'is_black_friday_period'
    ] = 1
    
    return df

# Применяем праздничные признаки
total_sales = create_holiday_features(total_sales)
wb_sales_processed = create_holiday_features(wb_sales_processed)
ozon_sales_processed = create_holiday_features(ozon_sales_processed)

print("✅ Праздничные признаки созданы")
print(f"\n📊 Статистика по праздникам:")
holiday_cols = ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window', 
                'is_mar_8_window', 'is_sep_1_window', 'is_black_friday_period']
print(total_sales[holiday_cols].sum())


In [ ]:
# Корреляция праздников с продажами
holiday_corr = []
for solo_code in all_solo_codes[:10]:  # Первые 10 товаров
    data = total_sales[total_sales['solo-code'] == solo_code]
    for holiday in holiday_cols:
        if data[holiday].sum() > 0:  # Если есть праздничные дни
            corr = data['Количество упак.'].corr(data[holiday])
            holiday_corr.append({
                'solo-code': solo_code,
                'holiday': holiday,
                'correlation': corr
            })

if holiday_corr:
    holiday_corr_df = pd.DataFrame(holiday_corr)
    print("\n📊 Корреляция праздников с продажами (примеры):")
    print(holiday_corr_df.groupby('holiday')['correlation'].mean().sort_values(ascending=False))

# Пример календарных признаков
print("\n📊 Пример календарных признаков:")
example_calendar = total_sales[
    ['Дата', 'day_of_week', 'is_weekend', 'month', 'is_new_year_period', 
     'is_feb_14_window', 'is_mar_8_window', 'is_black_friday_period']
].head(20)
print(example_calendar)


## 5. Обработка бизнес-ограничений

### 5.1. Товары на вывод (withdraw)
### 5.2. Дефектура (defecture)


In [ ]:
def apply_withdraw_constraint(sales_df, stocks_wb, stocks_ozon, stocks_our, withdraw_df):
    """Применение ограничений для товаров на вывод"""
    if len(withdraw_df) == 0:
        return sales_df.copy(), stocks_wb.copy(), stocks_ozon.copy(), stocks_our.copy()
    
    withdraw_codes = set(withdraw_df['solo-code'].unique())
    
    sales_df = sales_df.copy()
    stocks_wb = stocks_wb.copy()
    stocks_ozon = stocks_ozon.copy()
    stocks_our = stocks_our.copy()
    
    # Для товаров на вывод: продажи = 0, остатки равномерно распределяются
    for solo_code in withdraw_codes:
        # Продажи = 0
        sales_df.loc[sales_df['solo-code'] == solo_code, 'Количество упак.'] = 0
        
        # Равномерное распределение остатков до нуля
        for stocks_df in [stocks_wb, stocks_ozon, stocks_our]:
            mask = stocks_df['solo-code'] == solo_code
            if mask.sum() > 0:
                total_stock = stocks_df.loc[mask, 'Остаток'].iloc[-1]  # Последний остаток
                days_left = len(stocks_df.loc[mask])
                if days_left > 0:
                    daily_decrease = total_stock / days_left
                    stocks_df.loc[mask, 'Остаток'] = np.maximum(
                        0, 
                        total_stock - np.arange(days_left) * daily_decrease
                    )
    
    return sales_df, stocks_wb, stocks_ozon, stocks_our

# Применяем ограничения на вывод
total_sales, wb_stocks_processed, ozon_stocks_processed, our_stocks_processed = apply_withdraw_constraint(
    total_sales, wb_stocks_processed, ozon_stocks_processed, our_stocks_processed, withdraw
)

print("✅ Ограничения на вывод применены")
if len(withdraw) > 0:
    print(f"📦 Товаров на вывод: {len(withdraw)}")
    print(f"📦 Коды: {withdraw['solo-code'].unique()[:5]}")
else:
    print("📦 Товаров на вывод нет")


In [ ]:
def apply_defecture_constraint(sales_df, defecture_df):
    """Применение ограничений дефектуры"""
    if len(defecture_df) == 0:
        return sales_df.copy()
    
    sales_df = sales_df.copy()
    
    for idx, row in defecture_df.iterrows():
        solo_code = row['solo-code']
        end_date = row['Дата окончания дефектуры']
        
        if pd.isna(end_date):
            continue
        
        # В период до окончания дефектуры: продажи = 0
        mask = (sales_df['solo-code'] == solo_code) & (sales_df['Дата'] <= end_date)
        sales_df.loc[mask, 'Количество упак.'] = 0
    
    return sales_df

# Применяем ограничения дефектуры
total_sales = apply_defecture_constraint(total_sales, defecture)

print("✅ Ограничения дефектуры применены")
if len(defecture) > 0:
    print(f"⚠️ Товаров в дефектуре: {len(defecture)}")
    print(f"⚠️ Примеры:")
    print(defecture.head())
else:
    print("✅ Дефектуры нет")


## 6. Модели прогнозирования

Реализуем набор моделей для каждого solo-code:
- Baseline (Naive, Moving Average, Seasonal Naive)
- Для intermittent demand (Croston, SBA, TSB)
- Продвинутые (Linear Regression, ARIMA, SARIMA, SARIMAX, Prophet)


In [ ]:
# Baseline модели

def naive_forecast(data, horizon):
    """Naive: последнее значение"""
    last_value = data.iloc[-1]
    return [last_value] * horizon

def moving_average_forecast(data, window=7, horizon=30):
    """Moving Average"""
    ma_value = data[-window:].mean()
    return [ma_value] * horizon

def seasonal_naive_forecast(data, season_length=7, horizon=30):
    """Seasonal Naive: значение с прошлого сезона"""
    if len(data) < season_length:
        return [data.mean()] * horizon
    seasonal_values = data[-season_length:].values
    forecast = []
    for i in range(horizon):
        forecast.append(seasonal_values[i % season_length])
    return forecast

print("✅ Baseline модели определены")


In [ ]:
# Модели для intermittent demand

def croston_forecast(data, alpha=0.1, horizon=30):
    """Croston's method для прерывистого спроса"""
    if len(data) < 2:
        return [data.mean()] * horizon
    
    # Разделяем на ненулевые значения и интервалы
    non_zero = data[data > 0]
    if len(non_zero) < 2:
        return [non_zero.mean() if len(non_zero) > 0 else 0] * horizon
    
    # Экспоненциальное сглаживание размера заказа
    sizes = non_zero.values
    avg_size = sizes[0]
    for size in sizes[1:]:
        avg_size = alpha * size + (1 - alpha) * avg_size
    
    # Экспоненциальное сглаживание интервала
    intervals = []
    last_idx = 0
    for i in range(1, len(data)):
        if data.iloc[i] > 0:
            intervals.append(i - last_idx)
            last_idx = i
    
    if len(intervals) < 2:
        avg_interval = len(data) / max(len(non_zero), 1)
    else:
        avg_interval = intervals[0]
        for interval in intervals[1:]:
            avg_interval = alpha * interval + (1 - alpha) * avg_interval
    
    # Прогноз = средний размер / средний интервал
    forecast_demand = avg_size / max(avg_interval, 1)
    
    return [forecast_demand] * horizon

def sba_forecast(data, alpha=0.1, horizon=30):
    """Syntetos-Boylan Approximation"""
    croston_fc = croston_forecast(data, alpha, horizon)
    # SBA = Croston * (1 - alpha/2)
    return [x * (1 - alpha / 2) for x in croston_fc]

def tsb_forecast(data, alpha=0.1, beta=0.1, horizon=30):
    """Teunter-Syntetos-Babai method"""
    if len(data) < 2:
        return [data.mean()] * horizon
    
    non_zero = data[data > 0]
    if len(non_zero) < 2:
        return [non_zero.mean() if len(non_zero) > 0 else 0] * horizon
    
    # Размер заказа
    sizes = non_zero.values
    avg_size = sizes[0]
    for size in sizes[1:]:
        avg_size = alpha * size + (1 - alpha) * avg_size
    
    # Вероятность ненулевого спроса
    prob = (data > 0).sum() / len(data)
    last_prob = prob
    for i in range(1, len(data)):
        if data.iloc[i] > 0:
            last_prob = beta * 1 + (1 - beta) * last_prob
        else:
            last_prob = beta * 0 + (1 - beta) * last_prob
    
    forecast_demand = avg_size * last_prob
    return [forecast_demand] * horizon

print("✅ Модели для intermittent demand определены")


In [ ]:
# Продвинутые модели

def linear_regression_forecast(train_data, test_features, calendar_features):
    """Linear Regression с лагами и календарными признаками"""
    if len(train_data) < 10:
        return [train_data.mean()] * len(test_features)
    
    # Создание лагов
    lags = [1, 7, 14, 30]
    X_train = []
    y_train = []
    
    for i in range(max(lags), len(train_data)):
        features = []
        # Лаги
        for lag in lags:
            if i - lag >= 0:
                features.append(train_data.iloc[i - lag])
            else:
                features.append(0)
        # Календарные признаки
        if calendar_features is not None and i < len(calendar_features):
            cal_features = calendar_features.iloc[i][
                ['day_of_week', 'is_weekend', 'month', 'quarter', 
                 'month_sin', 'month_cos', 'dow_sin', 'dow_cos'] +
                ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window', 
                 'is_mar_8_window', 'is_black_friday_period']
            ].values
            features.extend(cal_features)
        else:
            features.extend([0] * 13)
        
        X_train.append(features)
        y_train.append(train_data.iloc[i])
    
    if len(X_train) == 0:
        return [train_data.mean()] * len(test_features)
    
    # Обучение модели
    try:
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Прогноз
        forecast = []
        last_values = train_data[-max(lags):].values.tolist()
        
        for i, test_row in enumerate(test_features.iterrows()):
            features = []
            # Лаги из последних значений
            for lag in lags:
                if len(last_values) >= lag:
                    features.append(last_values[-lag])
                else:
                    features.append(0)
            # Календарные признаки
            cal_features = test_row[1][
                ['day_of_week', 'is_weekend', 'month', 'quarter',
                 'month_sin', 'month_cos', 'dow_sin', 'dow_cos'] +
                ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window',
                 'is_mar_8_window', 'is_black_friday_period']
            ].values
            features.extend(cal_features)
            
            pred = model.predict([features])[0]
            forecast.append(max(0, pred))  # Негативные значения не допускаем
            last_values.append(pred)
        return forecast
    except:
        return [train_data.mean()] * len(test_features)

print("✅ Linear Regression модель определена")


In [ ]:
def arima_forecast(data, horizon=30, order=(1, 1, 1)):
    """ARIMA модель"""
    if len(data) < 10:
        return [data.mean()] * horizon
    
    try:
        # Убираем нули для стабильности
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        model = ARIMA(data_clean, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=horizon)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

def sarima_forecast(data, horizon=30, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7)):
    """SARIMA модель"""
    if len(data) < 20:
        return [data.mean()] * horizon
    
    try:
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        model = SARIMAX(data_clean, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        forecast = fitted_model.forecast(steps=horizon)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

def sarimax_forecast(data, exog_train, exog_test, horizon=30):
    """SARIMAX с экзогенными переменными (календарь)"""
    if len(data) < 20 or exog_train is None or exog_test is None:
        return [data.mean()] * horizon
    
    try:
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        # Выбираем ключевые календарные признаки
        exog_cols = ['is_weekend', 'month_sin', 'month_cos', 
                     'is_new_year_period', 'is_black_friday_period']
        exog_train_sel = exog_train[exog_cols] if all(c in exog_train.columns for c in exog_cols) else None
        exog_test_sel = exog_test[exog_cols] if all(c in exog_test.columns for c in exog_cols) else None
        
        if exog_train_sel is None or exog_test_sel is None:
            return sarima_forecast(data, horizon)
        
        model = SARIMAX(data_clean, exog=exog_train_sel, 
                       order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
        fitted_model = model.fit(disp=False)
        forecast = fitted_model.forecast(steps=horizon, exog=exog_test_sel)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

print("✅ ARIMA/SARIMA/SARIMAX модели определены")


In [ ]:
def prophet_forecast(data, dates, horizon_dates, holidays_df=None):
    """Prophet модель с праздниками"""
    if len(data) < 30:
        return [data.mean()] * len(horizon_dates)
    
    try:
        # Подготовка данных для Prophet
        prophet_data = pd.DataFrame({
            'ds': dates,
            'y': data.values
        })
        
        # Праздники для Prophet
        if holidays_df is None:
            holidays_list = []
            # Новый год
            for year in [2024, 2025]:
                holidays_list.append({'holiday': 'new_year', 'ds': f'{year}-12-31', 'lower_window': -15, 'upper_window': 10})
                holidays_list.append({'holiday': 'feb_14', 'ds': f'{year}-02-14', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'feb_23', 'ds': f'{year}-02-23', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'mar_8', 'ds': f'{year}-03-08', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'sep_1', 'ds': f'{year}-09-01', 'lower_window': -7, 'upper_window': 7})
            
            holidays_df = pd.DataFrame(holidays_list)
            holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])
        
        # Обучение модели
        model = Prophet(holidays=holidays_df, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
        model.fit(prophet_data)
        
        # Прогноз
        future = pd.DataFrame({'ds': horizon_dates})
        forecast = model.predict(future)
        
        return [max(0, x) for x in forecast['yhat'].values]
    except Exception as e:
        return [data.mean()] * len(horizon_dates)

print("✅ Prophet модель определена")


## 7. Валидация и выбор модели

Rolling backtesting с метриками: MAE, RMSE, MAPE


In [ ]:
def calculate_metrics(y_true, y_pred):
    """Расчет метрик прогнозирования"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # MAPE (устойчивый к нулям)
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = 0
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

def rolling_backtest(data, calendar_data, models_dict, train_size=180, test_size=30, step=30):
    """Rolling backtesting для всех моделей"""
    results = []
    
    if len(data) < train_size + test_size:
        return pd.DataFrame()
    
    for start_test in range(train_size, len(data) - test_size + 1, step):
        train_data = data.iloc[:start_test]
        test_data = data.iloc[start_test:start_test + test_size]
        
        train_calendar = calendar_data.iloc[:start_test] if calendar_data is not None else None
        test_calendar = calendar_data.iloc[start_test:start_test + test_size] if calendar_data is not None else None
        
        for model_name, model_func in models_dict.items():
            try:
                if model_name == 'Linear Regression':
                    forecast = model_func(train_data, test_calendar, train_calendar)
                elif model_name == 'SARIMAX':
                    forecast = model_func(train_data, train_calendar, test_calendar, len(test_data))
                elif model_name == 'Prophet':
                    train_dates = train_calendar['Дата'] if train_calendar is not None else pd.date_range('2024-01-01', periods=len(train_data), freq='D')
                    test_dates = test_calendar['Дата'] if test_calendar is not None else pd.date_range('2024-01-01', periods=len(test_data), freq='D')
                    forecast = model_func(train_data, train_dates, test_dates)
                else:
                    forecast = model_func(train_data, len(test_data))
                
                metrics = calculate_metrics(test_data.values, forecast[:len(test_data)])
                metrics['model'] = model_name
                metrics['fold'] = start_test
                results.append(metrics)
            except Exception as e:
                continue
    
    return pd.DataFrame(results)

print("✅ Функции валидации определены")


In [ ]:
# Тестирование моделей на примере товара
example_solo_code = all_solo_codes[0]
example_sales = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')
example_calendar = example_sales.copy()

# Подготовка данных
sales_series = example_sales['Количество упак.'].reset_index(drop=True)
demand_type = demand_df[demand_df['solo-code'] == example_solo_code]['demand_type'].iloc[0]

print(f"📦 Тестирование моделей для {example_solo_code}")
print(f"📊 Тип спроса: {demand_type}")
print(f"📊 Длина ряда: {len(sales_series)}")

# Выбор моделей в зависимости от типа спроса
models_to_test = {}

# Baseline всегда
models_to_test['Naive'] = lambda data, h: naive_forecast(data, h)
models_to_test['Moving Average'] = lambda data, h: moving_average_forecast(data, window=7, horizon=h)
models_to_test['Seasonal Naive'] = lambda data, h: seasonal_naive_forecast(data, season_length=7, horizon=h)

# Для intermittent
if demand_type == 'intermittent':
    models_to_test['Croston'] = lambda data, h: croston_forecast(data, horizon=h)
    models_to_test['SBA'] = lambda data, h: sba_forecast(data, horizon=h)
    models_to_test['TSB'] = lambda data, h: tsb_forecast(data, horizon=h)

# Продвинутые (если достаточно данных)
if len(sales_series) >= 30:
    models_to_test['Linear Regression'] = lambda data, cal, cal_train: linear_regression_forecast(data, cal, cal_train)
    models_to_test['ARIMA'] = lambda data, h: arima_forecast(data, horizon=h)
    
if len(sales_series) >= 60:
    models_to_test['SARIMA'] = lambda data, h: sarima_forecast(data, horizon=h)
    models_to_test['SARIMAX'] = lambda data, exog_train, exog_test, h: sarimax_forecast(data, exog_train, exog_test, h)
    
if len(sales_series) >= 90:
    models_to_test['Prophet'] = lambda data, dates, test_dates: prophet_forecast(data, dates, test_dates)

# Backtesting
if len(sales_series) >= 180:
    backtest_results = rolling_backtest(
        sales_series, 
        example_calendar,
        models_to_test,
        train_size=120,
        test_size=30,
        step=30
    )
    
    if len(backtest_results) > 0:
        print("\n📊 Результаты backtesting:")
        model_comparison = backtest_results.groupby('model').agg({
            'MAE': 'mean',
            'RMSE': 'mean',
            'MAPE': 'mean'
        }).round(2).sort_values('MAE')
        print(model_comparison)
        
        # Выбор лучшей модели
        best_model = model_comparison.index[0]
        print(f"\n🏆 Лучшая модель: {best_model}")
        print(f"   MAE: {model_comparison.loc[best_model, 'MAE']:.2f}")
        print(f"   RMSE: {model_comparison.loc[best_model, 'RMSE']:.2f}")
        print(f"   MAPE: {model_comparison.loc[best_model, 'MAPE']:.2f}%")
    else:
        print("⚠️ Недостаточно данных для backtesting")
else:
    print("⚠️ Недостаточно данных для полного backtesting (нужно минимум 180 дней)")


## 8. Rolling-прогноз продаж на 18 месяцев

Создаем прогноз для каждого solo-code на 18 месяцев вперед


In [ ]:
def generate_forecast(solo_code, sales_data, calendar_data, demand_type, models_dict, horizon_months=18):
    """Генерация прогноза для одного товара"""
    # Фильтрация данных
    item_data = sales_data[sales_data['solo-code'] == solo_code].sort_values('Дата')
    item_calendar = calendar_data[calendar_data['solo-code'] == solo_code].sort_values('Дата')
    
    if len(item_data) == 0:
        return None
    
    sales_series = item_data['Количество упак.'].reset_index(drop=True)
    
    # Создание календаря для прогноза
    last_date = item_data['Дата'].max()
    forecast_dates = pd.date_range(last_date + timedelta(days=1), periods=horizon_months * 30, freq='D')
    forecast_calendar = create_calendar_features(pd.DataFrame({'Дата': forecast_dates}))
    forecast_calendar = create_holiday_features(forecast_calendar)
    forecast_calendar['solo-code'] = solo_code
    
    # Выбор модели в зависимости от типа спроса и длины истории
    selected_model = None
    model_name = None
    
    # Логика выбора модели
    if demand_type == 'intermittent' and len(sales_series) >= 20:
        # Для прерывистого спроса
        if 'TSB' in models_dict:
            selected_model = lambda: tsb_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'TSB'
        elif 'Croston' in models_dict:
            selected_model = lambda: croston_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Croston'
        else:
            selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Moving Average'
    elif len(sales_series) >= 90:
        # Для длинной истории - Prophet или SARIMA
        if 'Prophet' in models_dict:
            train_dates = item_calendar['Дата'].values
            selected_model = lambda: prophet_forecast(sales_series, train_dates, forecast_dates)
            model_name = 'Prophet'
        elif 'SARIMA' in models_dict:
            selected_model = lambda: sarima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'SARIMA'
        else:
            selected_model = lambda: sarima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'SARIMA'
    elif len(sales_series) >= 30:
        # Для средней истории - ARIMA или Linear Regression
        if 'ARIMA' in models_dict:
            selected_model = lambda: arima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'ARIMA'
        elif 'Linear Regression' in models_dict:
            selected_model = lambda: linear_regression_forecast(sales_series, forecast_calendar, item_calendar)
            model_name = 'Linear Regression'
        else:
            selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Moving Average'
    else:
        # Для короткой истории - простые модели
        selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
        model_name = 'Moving Average'
    
    # Генерация прогноза
    try:
        forecast_values = selected_model()
        forecast_values = [max(0, x) for x in forecast_values]  # Негативные значения не допускаем
    except Exception as e:
        # Fallback к простой модели
        forecast_values = moving_average_forecast(sales_series, horizon=len(forecast_dates))
        model_name = 'Moving Average (fallback)'
    
    # Создание результата
    result = pd.DataFrame({
        'Дата': forecast_dates[:len(forecast_values)],
        'solo-code': solo_code,
        'Прогноз': forecast_values,
        'Модель': model_name
    })
    
    return result

print("✅ Функция генерации прогноза определена")


In [ ]:
# Генерация прогнозов для всех товаров
all_forecasts = []

print("📊 Генерация прогнозов для всех товаров...")
for idx, solo_code in enumerate(all_solo_codes):
    if (idx + 1) % 10 == 0:
        print(f"  Обработано: {idx + 1}/{len(all_solo_codes)}")
    
    demand_type = demand_df[demand_df['solo-code'] == solo_code]['demand_type'].iloc[0]
    
    # Прогноз для суммарных продаж
    forecast_total = generate_forecast(
        solo_code, 
        total_sales, 
        total_sales,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_total is not None:
        forecast_total['Канал'] = 'Total'
        all_forecasts.append(forecast_total)
    
    # Прогноз для WB
    forecast_wb = generate_forecast(
        solo_code,
        wb_sales_processed,
        wb_sales_processed,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_wb is not None:
        forecast_wb['Канал'] = 'WB'
        all_forecasts.append(forecast_wb)
    
    # Прогноз для Ozon
    forecast_ozon = generate_forecast(
        solo_code,
        ozon_sales_processed,
        ozon_sales_processed,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_ozon is not None:
        forecast_ozon['Канал'] = 'Ozon'
        all_forecasts.append(forecast_ozon)

if all_forecasts:
    forecasts_df = pd.concat(all_forecasts, ignore_index=True)
    print(f"\n✅ Прогнозы сгенерированы: {forecasts_df.shape}")
    print(f"📊 Уникальных товаров: {forecasts_df['solo-code'].nunique()}")
    print(f"📊 Каналов: {forecasts_df['Канал'].unique()}")
    print(f"\n📊 Пример прогноза:")
    print(forecasts_df.head(10))
else:
    print("⚠️ Прогнозы не сгенерированы")


In [ ]:
# Визуализация прогноза для примера товара
if 'forecasts_df' in locals() and len(forecasts_df) > 0:
    example_solo_code = all_solo_codes[0]
    
    # Фактические данные
    actual_total = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')
    actual_wb = wb_sales_processed[wb_sales_processed['solo-code'] == example_solo_code].sort_values('Дата')
    actual_ozon = ozon_sales_processed[ozon_sales_processed['solo-code'] == example_solo_code].sort_values('Дата')
    
    # Прогнозы
    forecast_total = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                  (forecasts_df['Канал'] == 'Total')].sort_values('Дата')
    forecast_wb = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                (forecasts_df['Канал'] == 'WB')].sort_values('Дата')
    forecast_ozon = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                 (forecasts_df['Канал'] == 'Ozon')].sort_values('Дата')
    
    fig, axes = plt.subplots(3, 1, figsize=(16, 12))
    
    # Total
    axes[0].plot(actual_total['Дата'], actual_total['Количество упак.'], 
                 label='Факт', linewidth=2, color='blue')
    if len(forecast_total) > 0:
        axes[0].plot(forecast_total['Дата'], forecast_total['Прогноз'], 
                    label='Прогноз', linewidth=2, color='red', linestyle='--')
        axes[0].axvline(x=actual_total['Дата'].max(), color='gray', linestyle=':', alpha=0.7, label='Начало прогноза')
    axes[0].set_title(f'Прогноз продаж: {example_solo_code} - Total', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Дата')
    axes[0].set_ylabel('Количество упаковок')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # WB
    axes[1].plot(actual_wb['Дата'], actual_wb['Количество упак.'], 
                 label='Факт WB', linewidth=2, color='blue')
    if len(forecast_wb) > 0:
        axes[1].plot(forecast_wb['Дата'], forecast_wb['Прогноз'], 
                    label='Прогноз WB', linewidth=2, color='red', linestyle='--')
        axes[1].axvline(x=actual_wb['Дата'].max(), color='gray', linestyle=':', alpha=0.7)
    axes[1].set_title(f'Прогноз продаж: {example_solo_code} - Wildberries', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Дата')
    axes[1].set_ylabel('Количество упаковок')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Ozon
    axes[2].plot(actual_ozon['Дата'], actual_ozon['Количество упак.'], 
                 label='Факт Ozon', linewidth=2, color='blue')
    if len(forecast_ozon) > 0:
        axes[2].plot(forecast_ozon['Дата'], forecast_ozon['Прогноз'], 
                    label='Прогноз Ozon', linewidth=2, color='red', linestyle='--')
        axes[2].axvline(x=actual_ozon['Дата'].max(), color='gray', linestyle=':', alpha=0.7)
    axes[2].set_title(f'Прогноз продаж: {example_solo_code} - Ozon', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Дата')
    axes[2].set_ylabel('Количество упаковок')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Использованная модель: {forecast_total['Модель'].iloc[0] if len(forecast_total) > 0 else 'N/A'}")


## 9. Расчет страхового запаса

Формула: Safety Stock = Z * σ * sqrt(lead_time)

где:
- Z = 1.65 для service level 95%
- σ = стандартное отклонение дневного спроса
- lead_time = время поставки (по умолчанию 30 дней)


In [ ]:
def calculate_safety_stock(sales_data, service_level=0.95, lead_time=30):
    """Расчет страхового запаса для всех товаров"""
    # Z-score для service level
    from scipy import stats
    z_score = stats.norm.ppf(service_level)
    
    safety_stocks = []
    
    for solo_code in all_solo_codes:
        item_data = sales_data[sales_data['solo-code'] == solo_code].sort_values('Дата')
        
        if len(item_data) == 0:
            continue
        
        demand = item_data['Количество упак.'].values
        
        # Средний дневной спрос
        mean_demand = np.mean(demand)
        
        # Стандартное отклонение дневного спроса
        std_demand = np.std(demand)
        
        # Страховой запас
        safety_stock = z_score * std_demand * np.sqrt(lead_time)
        safety_stock = max(0, safety_stock)  # Не может быть отрицательным
        
        safety_stocks.append({
            'solo-code': solo_code,
            'mean_demand': mean_demand,
            'std_demand': std_demand,
            'safety_stock': safety_stock,
            'lead_time': lead_time,
            'service_level': service_level
        })
    
    return pd.DataFrame(safety_stocks)

# Расчет страхового запаса
safety_stock_df = calculate_safety_stock(total_sales, service_level=0.95, lead_time=30)

print("📊 Страховой запас рассчитан:")
print(safety_stock_df.head(10))
print(f"\n📊 Общая статистика:")
print(safety_stock_df[['mean_demand', 'std_demand', 'safety_stock']].describe())


## 10. Планирование отгрузок

На основе:
- Прогноза продаж
- Остатков на складах (WB, Ozon, наш склад)
- Страхового запаса
- Количества упаковок в коробе (count_box)

Рассчитываем:
- Чистую потребность
- Отгрузку в упаковках и коробах


In [ ]:
def plan_shipments(forecasts_df, stocks_wb, stocks_ozon, stocks_our, safety_stock_df, count_box_df, horizon_days=30):
    """Планирование отгрузок"""
    shipment_plan = []
    
    # Создание словаря count_box
    count_box_dict = dict(zip(count_box_df['solo-code'], count_box_df['Кол-во']))
    
    # Получаем последние остатки
    last_date = max(
        stocks_wb['Дата'].max(),
        stocks_ozon['Дата'].max(),
        stocks_our['Дата'].max()
    )
    
    last_stocks_wb = stocks_wb[stocks_wb['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    last_stocks_ozon = stocks_ozon[stocks_ozon['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    last_stocks_our = stocks_our[stocks_our['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    
    # Словарь страхового запаса
    safety_stock_dict = dict(zip(safety_stock_df['solo-code'], safety_stock_df['safety_stock']))
    
    # Обработка прогнозов по каналам
    for channel in ['WB', 'Ozon']:
        channel_forecasts = forecasts_df[
            (forecasts_df['Канал'] == channel) & 
            (forecasts_df['Дата'] <= last_date + timedelta(days=horizon_days))
        ].copy()
        
        for solo_code in channel_forecasts['solo-code'].unique():
            item_forecast = channel_forecasts[channel_forecasts['solo-code'] == solo_code].sort_values('Дата')
            
            # Текущие остатки
            stock_wb = last_stocks_wb.get(solo_code, 0)
            stock_ozon = last_stocks_ozon.get(solo_code, 0)
            stock_our = last_stocks_our.get(solo_code, 0)
            
            # Страховой запас
            safety_stock = safety_stock_dict.get(solo_code, 0)
            
            # Количество в коробе
            boxes_per_pack = count_box_dict.get(solo_code, 1)
            
            # Прогноз продаж на период
            forecast_demand = item_forecast['Прогноз'].sum()
            
            # Текущий остаток на канале
            if channel == 'WB':
                current_stock = stock_wb
            else:
                current_stock = stock_ozon
            
            # Чистая потребность = прогноз + страховой запас - текущий остаток
            net_requirement = forecast_demand + safety_stock - current_stock
            net_requirement = max(0, net_requirement)  # Не может быть отрицательным
            
            # Округление до целого короба
            shipment_boxes = np.ceil(net_requirement / boxes_per_pack) if boxes_per_pack > 0 else 0
            shipment_packs = shipment_boxes * boxes_per_pack
            
            shipment_plan.append({
                'Дата_планирования': last_date,
                'solo-code': solo_code,
                'Канал': channel,
                'Прогноз_продаж': forecast_demand,
                'Остаток_WB': stock_wb,
                'Остаток_Ozon': stock_ozon,
                'Остаток_наш': stock_our,
                'Страховой_запас': safety_stock,
                'Чистая_потребность': net_requirement,
                'Отгрузка_упаковок': shipment_packs,
                'Отгрузка_коробов': shipment_boxes,
                'Упаковок_в_коробе': boxes_per_pack
            })
    
    return pd.DataFrame(shipment_plan)

# Планирование отгрузок
if 'forecasts_df' in locals() and len(forecasts_df) > 0:
    shipment_plan_df = plan_shipments(
        forecasts_df,
        wb_stocks_processed,
        ozon_stocks_processed,
        our_stocks_processed,
        safety_stock_df,
        count_box,
        horizon_days=30
    )
    
    print("📦 План отгрузок создан:")
    print(shipment_plan_df.head(20))
    print(f"\n📊 Сводка по отгрузкам:")
    print(shipment_plan_df.groupby('Канал').agg({
        'Отгрузка_упаковок': 'sum',
        'Отгрузка_коробов': 'sum'
    }).round(2))
else:
    print("⚠️ Прогнозы не найдены, план отгрузок не создан")


In [ ]:
print("=" * 80)
print("📊 ИТОГОВАЯ СВОДКА СИСТЕМЫ ПРОГНОЗИРОВАНИЯ")
print("=" * 80)

print(f"\n📦 Обработано товаров: {len(all_solo_codes)}")
print(f"📅 Период данных: {min_date.date()} - {max_date.date()}")
print(f"📅 Всего дней в истории: {(max_date - min_date).days + 1}")

print(f"\n📊 Классификация спроса:")
if 'demand_df' in locals():
    print(demand_df['demand_type'].value_counts())

print(f"\n📈 Использованные модели:")
if 'forecasts_df' in locals():
    model_usage = forecasts_df.groupby('Модель').size().sort_values(ascending=False)
    print(model_usage)

print(f"\n📦 Планирование отгрузок:")
if 'shipment_plan_df' in locals():
    total_shipment = shipment_plan_df.groupby('Канал').agg({
        'Отгрузка_упаковок': 'sum',
        'Отгрузка_коробов': 'sum',
        'solo-code': 'nunique'
    })
    print(total_shipment)

print(f"\n🏬 Страховой запас:")
if 'safety_stock_df' in locals():
    print(f"  Средний страховой запас: {safety_stock_df['safety_stock'].mean():.2f} упаковок")
    print(f"  Медианный страховой запас: {safety_stock_df['safety_stock'].median():.2f} упаковок")
    print(f"  Максимальный страховой запас: {safety_stock_df['safety_stock'].max():.2f} упаковок")

print("\n✅ Система прогнозирования успешно построена!")


In [ ]:
print("=" * 80)
print("⚠️ РИСКИ И РЕКОМЕНДАЦИИ")
print("=" * 80)

print("\n🔴 РИСКИ:")
print("1. Качество прогноза зависит от длины исторических данных")
print("   - Для товаров с короткой историей (<30 дней) используются простые модели")
print("   - Рекомендация: накапливать данные минимум 3-6 месяцев")
print("\n2. Прерывистый спрос (intermittent) сложен для прогнозирования")
print("   - Высокая доля нулевых дней снижает точность")
print("   - Рекомендация: использовать специализированные модели (Croston, TSB)")
print("\n3. Внешние факторы не учтены")
print("   - Промоакции, изменения цен, конкурентная среда")
print("   - Рекомендация: добавить внешние данные при наличии")
print("\n4. Дефектура и вывод товаров")
print("   - Могут существенно влиять на прогноз")
print("   - Рекомендация: регулярно обновлять справочники")
print("\n5. Сезонность")
print("   - Модели учитывают базовую сезонность, но могут не улавливать специфику категории")
print("   - Рекомендация: настраивать сезонность под категорию товара")

print("\n🟢 РЕКОМЕНДАЦИИ ПО РАЗВИТИЮ:")
print("1. Автоматизация обновления данных")
print("   - Ежедневная загрузка новых данных")
print("   - Автоматический пересчет прогнозов")
print("\n2. Мониторинг качества прогноза")
print("   - Отслеживание метрик (MAE, RMSE, MAPE) в реальном времени")
print("   - Алерты при значительном отклонении")
print("\n3. A/B тестирование моделей")
print("   - Сравнение разных моделей на новых данных")
print("   - Автоматический выбор лучшей модели")
print("\n4. Интеграция с внешними данными")
print("   - Промоакции, изменения цен")
print("   - Экономические индикаторы")
print("   - Данные конкурентов (если доступны)")
print("\n5. Мультиуровневое прогнозирование")
print("   - Прогноз на уровне категории + корректировка на уровне SKU")
print("   - Учет взаимосвязей между товарами")
print("\n6. Оптимизация запасов")
print("   - Учет ограничений по складам")
print("   - Минимизация общих затрат (хранение + дефицит)")
print("\n7. Визуализация и дашборды")
print("   - Интерактивные графики прогнозов")
print("   - Анализ отклонений")
print("   - Планы отгрузок в удобном формате")

print("\n✅ Система готова к использованию!")
